Nesses notebooks temos os seguintes códigos:

1. Extração dos links dos PDFS Curtos: A LLM lê os pdfs curtos no volume e extrai os links. Depois, é feita uma verificação dos links que estão funcionando.
2. Criação da tabela "extracao_prospectos_kinea" para armazenar as informações para a clusterização
3. Código para a extração das informações dos prospectos KINEA usando LLM. Os prompts estão específicos pros formatos dos documentos da Kinea
4. Criação da tabela "extracao_prospectos" para armazenas as informações para a clusterização
4. Código para a extração das informações dos prospectos não Kinea.

In [0]:
%pip install PyPDF2 pymupdf openai pandas pdfplumber 


In [0]:
pip install openai==0.28

In [0]:
%restart_python

1. Código para extrair os links dos PDFS curtos:

In [0]:
import fitz  # PyMuPDF
import os
import openai
import base64

# Extrair os links dos pdfs curtos (todos)

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

DEPLOYMENT_NAME = "gpt-4.1-mini"

# Caminho com os PDFs curtos
BASE_DIR = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"

def merge_text_blocks(entries):
    merged = []
    buf = []

    def flush_buf():
        if not buf:
            return
        merged.append({
            "type": "text",
            "text": "\n".join(buf).strip()
        })
        buf.clear()

    for e in entries:
        if e["type"] == "text":
            buf.append(e["text"])
        else:
            flush_buf()
            merged.append(e)
    flush_buf()
    return merged

def create_messages_for_pdf(pdf_path):
    results = []
    doc = fitz.open(pdf_path)

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b["type"] == 0:
                lines = b.get("lines", [])
                txt_lines = []
                for line in lines:
                    spans = line.get("spans", [])
                    line_text = "".join(span.get("text", "") for span in spans)
                    if line_text:
                        txt_lines.append(line_text)
                full_text = "\n".join(txt_lines).strip()
                if full_text:
                    results.append({"type": "text", "text": full_text})

    doc.close()
    return merge_text_blocks(results)

def extract_links_from_pdf(pdf_path):
    try:
        messages = [
            {
                "role": "system",
                "content": """
                Você é um assistente que ajuda a identificar links presentes em documentos de texto.

                Sua tarefa é:
                – Analisar o conteúdo textual de um PDF (como um prospecto, regulamento ou aviso).
                – Procurar **todos os links** de documentos do tipo PDF ou DOC (como regulamentos, lâminas, anúncios, planilhas, etc).
                – Os links podem estar em qualquer formato: encurtados, integrais, com ou sem “https”.
                – O retorno deve conter SOMENTE os links (1 por linha), já formatados corretamente (iniciando com http ou https).
                – Se não encontrar nenhum link, retorne apenas a frase: `link não encontrado`.
                """
            },
            {
                "role": "user",
                "content": create_messages_for_pdf(pdf_path)
            }
        ]

        completion = openai.ChatCompletion.create(
            engine=DEPLOYMENT_NAME,
            messages=messages,
            temperature=0.0
        )

        response = completion.choices[0].message['content']
        return response.strip()

    except Exception as e:
        return f"❌ Erro ao processar '{os.path.basename(pdf_path)}': {str(e)}"

def list_all_pdf_paths(base_dir):
    try:
        files = dbutils.fs.ls(base_dir)
        pdfs = [
            os.path.join(base_dir, f.name)
            for f in files
            if f.name.lower().endswith(".pdf")
        ]
        return pdfs
    except Exception as e:
        print(f"Erro ao listar PDFs: {str(e)}")
        return []

# Execução principal
print("🔍 Extraindo links de TODOS os PDFs do volume:\n")
pdf_paths = list_all_pdf_paths(BASE_DIR)

if not pdf_paths:
    print("Nenhum PDF encontrado.")
else:
    for i, pdf_path in enumerate(pdf_paths, 1):
        file_name = os.path.basename(pdf_path)
        

        result = extract_links_from_pdf(pdf_path)
        print(f"{file_name}:\n{result}\n")


        


In [0]:
import requests

# Dos links extraídos acima, verifica quais estão funcionandos e se são documentos PDF

def verificar_links(links):
    resultados = {}

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    for link in links:
        try:
            response = requests.head(link, allow_redirects=True, timeout=10, headers=headers)
            content_type = response.headers.get("Content-Type", "").lower()

            if response.status_code >= 400:
                continue  # Ignora links inválidos
            elif "pdf" in content_type or "msword" in content_type or "officedocument.wordprocessingml.document" in content_type:
                resultados[link] = "✅ Documento válido"
            # Senão, ignora também
        except requests.exceptions.RequestException:
            continue  # Ignora links com erro

    return resultados

# Exemplo de uso
links_testar = [
    "http://www.personaltrader.com.br/documentos/fidc_empresarial/prospcecto_empresarial_050106.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empresarial/prospcecto_empresarial_040406.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empresarial/prospecto_empresarial_220506.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empresarial/prospecto_empresarial_120906.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empresarial2/prospecto_empresarial2_030807.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empresarial3/prospecto_empresarial_011107.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empresarial3/prospecto_empresarial_160308.pdf",
"http://www.personaltrader.com.br/documentos/fidc_master/prospcecto_master_050106.pdf",
"http://www.personaltrader.com.br/documentos/fidc_master/prospecto_master_200406.pdf",
"http://www.personaltrader.com.br/documentos/fidc_master/prospecto_master_260506.pdf",
"http://www.personaltrader.com.br/documentos/fidc_master/prospecto_master_080307.pdf",
"http://www.personaltrader.com.br/documentos/fidc_master/prospecto_master_160407.pdf",
"http://www.personaltrader.com.br/documentos/fidc_master/prospecto_master_040509.pdf",
"http://www.verax.com.br/fundos/prospecto_fidc_multicred.pdf",
"http://www.cetip.com.br/fundos_v06/prospectos/06-11-29%20-%20MULTICRED%20-%20PROSPECTO%202ª%20SÉRIE%20(V2).PDF",
"http://www.bcsul.com.br/documentos/investimentos/FIDC_RPW_Prospecto_exig_CVM.pdf",
"http://www.verax.com.br/Downloads/FIDC_RPW_Prospecto_exig_CVM.pdf",
"http://www.personaltrader.com.br/documentos/fidc_jcpsul/prospecto_jcpsul_180407.pdf",
"http://www.personaltrader.com.br/documentos/fidc_jcpsul/prospecto_jcpsul_170707.pdf",
"http://www.personaltrader.com.br/documentos/fidc_hope/prospecto_hope_120307.pdf",
"http://www.personaltrader.com.br/documentos/fidc_hope/prospecto_hope_250507.pdf",
"http://www.oliveiratrust.com/port/fds/doc//31/POLO%20FIDC%20-%20Prospecto%20Preliminar%2008%2001%202007.pdf",
"http://www.oliveiratrust.com/port/fds/doc//31/POLO%20FIDC%20-%20Prospecto%20Definitivo%2030%2001%202007.pdf",
"http://www.oliveiratrust.com/port/fds/doc//31/POLO%20FIDC%20-%20Prospecto%20Definitivo%2028%2003%202007.pdf",
"https://www.gradualcorretora.com.br/compartilhado/arquivos/Prospecto_Limpo_26.06.pdf",
"http://www.personaltrader.com.br/documentos/fidc_redfactor/prospecto_redfactor_130407.pdf",
"http://www.personaltrader.com.br/documentos/fidc_redfactor/prospecto_redfactor_130707.pdf",
"http://www.personaltrader.com.br/documentos/fidc_valecred/prospecto_valecred_080507.pdf",
"http://www.personaltrader.com.br/documentos/fidc_valecred/prospecto_valecred_170707.pdf",
"http://www.personaltrader.com.br/documentos/fidc_valecred/prospecto_valecred_051107.pdf",
"http://www.nsgcapital.com.br/arquivos/fundos/Prospecto%20FIDC%20Brazil%20Plus%20-%2020101230.pdf",
"http://www.nsgcapital.com.br/arquivos/fundos/Prospecto%20FIDC%20Brazil%20Plus%20-%2002.03.11.pdf",
"http://www.nsgcapital.com.br/arquivos/fundos/Prospecto%20FIDC%20Brazil%20Plus%20-%20(15.06.2011)%20vf.pdf",
"http://www.nsgcapital.com.br/arquivos/cvm/Prospecto_FIDC_Brazil_Plus_20120229.pdf",
"http://www.oliveiratrust.com/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%2001%2006%202007.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%2031%2008%202007.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%2026%2011%202008.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%2006%2005%202009.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%2026%2010%202009.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%202a%20Distribuicao%20de%20Quotas_04%2011%202010.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20SupplieCard%20_%20Prospecto%202a%20Distribuicao%20de%20Quotas_26%2007%202011.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/37/Ourinvest%20Suppliercard%20_%20Prospecto%202a%20Distribuicao%20de%20Quotas_02%2012%202011_pos%20Modif%20Of%20Nov11.pdf",
"http://www.luzpublicidade.com.br/admin/temp/ftp/TRADEMAXDEF.pdf",
"http://www.personaltrader.com.br/documentos/fidc_prospecta/prospecto_prospecta_211207.pdf",
"http://www.personaltrader.com.br/documentos/fidc_prospecta2/prospecto_prospecta2_070308.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/48/UNION%20NATIONAL%20AGRO%20+%20FIDC%20FINAC%20AGROPEC_Prospecto%20Definitivo%2005%2009%202007.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/48/Prospecto_Union%20Agro_exigenciasCVM_05-11-07%20_Exigencias%20CVM%20SM%20-%20Final_.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/48/UNION%20NATIONAL%20AGRO%20+%20FIDC%20FINAC%20AGROPEC_Prospecto%20Definitivo%2011%2012%202007.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/48/UNION%20NATIONAL%20AGRO%20+%20FIDC%20FINAC%20AGROPEC_Prospecto%20Definitivo%2018%2004%202008.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/48/Union%20National%20+%20Agro_Prospecto%20Definitivo%20-%2024%2007%202008.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/48/Imprimir%20PROSPECTO%20UNION%20NATIONAL%20AGRO.tif%20_(242%20páginas_).pdf",
"http://www.verax.com.br/downloads/Prospecto_FIDC.TRENDBANK.CREDITMIX.pdf",
"http://www.cetip.com.br/fundos_v06/prospectos/PROSPECTO_FIDC_TRENDBANK.pdf",
"http://www.bcsul.com.br/PROSPECTO.FIDC.TRENDBANK.CREDITMIX.pdf",
"http://www.cetip.com.br/fundos_v06/prospectos/TRENDBANK_CREDITMIX_PROSPECTO_VFINAL.pdf",
"http://www.verax.com.br/Downloads/Regulamento_FIDC.Trendbank.CreditMixL.pdf",
"http://www.bancopetra.com.br/documentos/multi_recebiveis_ii_fidc_prospecto_9_serie_-_quotas.pdf",
"http://www.petracorretora.com.br/wp-content/uploads/2014/09/Prospecto-SubPreferencial-D_20160224.pdf",
"http://www.personaltrader.com.br/documentos/fidc_asia/prospecto_asia_100408.pdf",
"http://www.personaltrader.com.br/documentos/fidc_asia/prospecto_asia_300508.pdf",
"http://www.personaltrader.com.br/documentos/fidc_radice/prospecto_radice_020408.pdf",
"http://www.personaltrader.com.br/documentos/fidc_radice/prospecto_radice_300508.pdf",
"http://www.personaltrader.com.br/documentos/fidc_radice/prospecto_radice_120808.pdf",
"http://www.personaltrader.com.br/documentos/fidc_sm/Prospecto_FIDC_SM_080708.pdf",
"http://www.personaltrader.com.br/documentos/fidc_daniele/prospecto_daniele_180608.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_20101125_Prospecto_FIDC_GMAC_WS.pdf",
"http://www.oliveiratrust.com.br/port/fds/doc/64/FIDC%20Mercantis%20Agro%20MS_Prospecto%20Definitivo_24%2012%202008.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_20081117_URGENTEPROSPECTOFIDCQUATA.pdf",
"http://www.cdinvest.com.br/arquivos/Prospecto_30122008.pdf",
"http://www.hsbc.com.br/1/PA_1_1_S5/content/hbbr_pws/pt/para-voce/investimentos/fundos-de-investimento/fundos-investimento-direitos-creditorios-fidc/fidc-petrobras/docs/prosp_petrobras.pdf",
"http://www.planner.com.br/prosp_petrobras.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_20110112_BBIF_MASTER_FIDC%20LP_%20PROSPECTO_.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS20111212_Prospecto_QTIPCAFIDC_JUROS_REAL.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_16_11_2012_QT_IPCA_FIDC_Prospecto_27_09_2012.pdf",
"http://www.gradualinvestimentos.com.br/pdfs/PROSPECTO_EXODUS_MASTER_130212.pdf",
"http://www.gradualinvestimentos.com.br/pdfs/20120326-EXODUS-MASTER-PROSPECTO.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_20110207_Atualizacao_FIDC.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_07052012_LEME_PROSPECTO.pdf",
"http://www.gradualinvestimentos.com.br/Resc/Upload/PDFs/LEME_M_IPCA_FIDC_Prosp2.pdf",
"http://downloads.caixa.gov.br/_arquivos/investidores/vincicreditoedesenvolvimentoifidc/prospecto_definitivo_Vinci_cd_i_fidc.pdf",
"https://www.gradualinvestimentos.com.br/Resc/Upload/PDFs/2015-01/ProspectoEGoalOne.pdf",
"http://www.petracorretora.com.br/documentos/prospecto-definitivo-empirica-sifra-premium-2sr.pdf",
"https://www.btgpactual.com/home/docs/Arquivo/FIDCMITSUBPROSPDEF2011.pdf",
"http://gradualinvestimentos.com.br/pdfs//EXODUS%20INSTITUCIONAL_PROSPECTO.pdf",
"http://www.gradualinvestimentos.com.br/Resc/Upload/PDFs/PROSPECTO_FIDC_EXODUS_INSTITUCIONAL_280912.pdf",
"http://www.gradualinvestimentos.com.br/Resc/Upload/PDFs/Prospecto_FIDC_Exodus_Institucional-1412.pdf",
"http://www.gradualinvestimentos.com.br/Resc/Upload/PDFs/Prospecto_ExodusInstitucional_10.02.2014.pdf",
"http://www.personaltrader.com.br/documentos/fidc_empirica_sifra/prospecto_fidcsifrastar_090312.pdf",
"http://www.petracorretora.com.br/documentos/prospecto/Prospecto_Sifra_Star_Senior.pdf",
"http://www.petracorretora.com.br/documentos/prospecto-definitivo-empirica-sifra-star.pdf",
"http://corretora.finaxis.com.br/wp-content/uploads/2014/09/prospecto-definitivo-empirica-sifra-star-1.pdf",
"http://www.bb.com.br/docs/pub/siteEsp/sitedtvm/dwn/prospectofidc.pdf",
"http://vx.vortx.com.br/Upload/EncontrarArquivoPublico?nomeDocumento=FIDC%20NP%20-%20CROWN%20OCEAN%20-%2018676119%20-%20Prospecto%20-%2020200325.pdf&codigoDocumento=78923",
"http://www.bancopetra.com.br/wp-content/uploads/2014/12/prosp_FIDC-Lavoro_20012015_vers%C3%A3o-completa.pdf",
"http://www.petracorretora.com.br/wp-content/uploads/2014/12/prosp_FIDC-Lavoro_20012015_vers%C3%A3o-completa.pdf",
"https://www.socopa.com.br/Arquivo/prospecto_multiplica_limpa.pdf",
"https://www.socopa.com.br/Arquivo/GII_GestaoInteligente_Prospecto.pdf",
"https://ww69.itau.com.br/fileserver/relatorios/Prospecto_FIC_FIDC_Kinea_Infra.pdf",
"https://www.itau.com.br/_arquivosestaticos/itauBBA/Prospectos/Prospecto_FIC-FIDC_Kinea_Infra.PDF",
"http://www.concordia.com.br/mkt/FIDC/novos_arquivos/Anga_Sabemi_ConsignadosVII/PROSPECTO_FIDC_ANGA_SABEMI_VII_19_04_2017.pdf",
"http://www.concordia.com.br/mkt/FIDC/novos_arquivos/Anga_Sabemi_ConsignadosVII/PROSPECTO_FIDC_ANGA_SABEMI_VII_131017.pdf",
"http://www.concordia.com.br/mkt/FIDC/novos_arquivos/Anga_Sabemi_ConsignadosVII/PROSPECTO_FIDC_ANGA_SABEMI_VII_170418.pdf",
"http://www.concordia.com.br/mkt/FIDC/novos_arquivos/Anga_Sabemi_ConsignadosVII/PROSPECTO_FIDC_ANGA_SABEMI_VII_290818.pdf",
"http://www.winnerpublicidade.com/pdf/sabemi/180619/Comunicado%20de%20Suspensao%20e%20Prospecto.pdf",
"https://bemdtvm.bradesco/Upload%20Documents/Funds/3028/2/Prospecto_Cid_2_2017 105165716138.pdf",
"http://www.patriainvestimentos.com.br/downloads/p/Prospectos/Pátria%20Brazilian%20Private%20Equity%20Fund%20III%20-%20FIP.pdf",
"http://www.luzpublicidade.com.br/admin/temp/ftp/RIVIERADEFSITE.pdf",
"http://clientes.luzpublicidadesp.com.br/RIVIERASITEPORT2.pdf",
"http://downloads.caixa.gov.br/_arquivos/investidores/fip_modal/Prospecto_FIP_Oleo_e_Gas_v14dez09_com_dados_CVM.pdf",
"http://www.nsgcapital.com.br/arquivos/fundos/FIP%20TRISCORP%2009.08.2010.pdf",
"http://www.nsgcapital.com.br/arquivos/fundos/TRISCORPDEFCVM.pdf",
"http://negocios.socopa.com.br/Arquivo/P2BrasilInfraestrutura_Prospecto%20Preliminar.pdf",
"http://www.patriainvestimentos.com.br/Upload/Prospecto_Definitivo_-_Brazilian_Private_Equity_IV_FIQFIP_-_Segunda_Emissao.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_20111216_BTG_InfraestruturaII_Prospecto.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_24_01_2013_PROSPECTO_PRELIMINAR_23_01_2013.pdf",
"https://www.brasil.citibank.com/JPS/content/pdf/ICMS_19_02_2013_BTGINFRA_DEF.pdf",
"http://www.petracorretora.com.br/documentos/prospecto/Innova-FIP-Prospecto.pdf",
"http://www.bancopetra.com.br/documentos/prospecto/Innova-FIP-Prospecto.pdf",
"https://www.cshg.com.br/site/publico/download/fundos/imob/pdf_imob/CSHG_RealtyDevelopmentFIP/ProspectoDefinitivo_13_03_07_reduzido.pdf",
"https://www.cshg.com.br/site/publico/download/fundos/imob/pdf_imob/CSHG_RealtyDevelopmentFIP/ProspectoDefinitivo_13_09_20.pdf",
"http://static.gerafuturo.com.br/Documentos/Prospecto_-_P2_Brasil_Infraestrutura_III_FIQFIP.pdf",
"http://www.p2brasil.com.br/p2brasil//News/Prospecto_P2_Brasil_Infraestrutura_III_FIQFIP_2_emissao.pdf",
"http://www.daycoval.com.br/includes/pdf/BioestFipProspectoDefinitivo.pdf",
"https://www.bancovotorantim.com.br/web/export/sites/bancovotorantim/bvarquivos/ofertas/2016/Prospecto-Definitivo-nov2016.pdf",
"http://clientes.luzsp.com.br/clientes/VRE/VRE_DESENVOLVIMENTO.pdf",
"http://www.lionstrust.com.br/wa_files/20171107_20-_20Prospecto_20Definitivo_20FIP_20Kinea_20PE_20IV_20Institucional_20I.PDF",
"http://vx.vortx.com.br/Upload/EncontrarArquivoPublico?nomeDocumento=XP%20Infra%20II%20-%202%C2%AA%20Emiss%C3%A3o%20(400)%20-%20Protocolo%20P%C3%B3s%20Registro%20-%20Prospecto%20(10-Fev).pdf&codigoDocumento=78141",
"http://vx.vortx.com.br/Upload/EncontrarArquivoPublico?nomeDocumento=Prospecto%20Preliminar%20(limpo).pdf&codigoDocumento=81167",
"https://cloud.luzcapitalmarkets.com.br/.FIP_XP_INFRA_II_4-EMISSAO_DEF.pdf",
"http://www.vincipartners.com/uploads/VCP%20III%20FIP%20M%20II%20-%20Prospecto%20v.%20limpa.pdf",
"https://www.itau.com.br/_arquivosestaticos/Itau/Private/aba-fundos-de-investimento/Itau_Feeder_Vinci_III_--_Prospecto_Definitivo.pdf",
"https://static.btgpactual.com/media/anexo-g1-prospecto-preliminar-versao-diagramada-19112020.pdf",
"https://www.bancoplural.com/Files/FundosAdm/270_Prospecto_Definitivo.pdf",
"https://static.btgpactual.com/media/fip-ie-perfin-prospecto-definitivo-diagramado.pdf",
"https://www.modalasset.com.br/wp-content/uploads/2020/02/1.-Prospecto-Definitivo.pdf",
"https://static.btgpactual.com/media/btg-infra-def.pdf",
"https://static.btgpactual.com/media/fip-econo-real-prospecto-definitivo-31012020.pdf",
"https://static.btgpactual.com/media/anexo-d1-20201027-btg-fip-economia-real-prospecto-preliminar-diagramado.pdf",
"https://static.btgpactual.com/media/btg-economia-real-def.pdf",
"https://static.btgpactual.com/media/1-prot-ix-prisma-proton-fip-ie-prospecto-preliminar.pdf",
"https://static.btgpactual.com/media/prospecto-definitivo-fip-ie-endurance-debt-11012021.pdf",
"https://static.btgpactual.com/media/prospecto/36642497000199_20210317_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/36642570000122_20210419_PRO.pdf",
"http://clientes2.luzsp.com.br/clientes/XP_SELECTION_PREL.pdf",
"https://static.btgpactual.com/media/prospecto/40011391000164_20210705_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/40011391000164_20211004_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/41082947000176_20210721_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/42120193000164_20211229_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/42120193000164_20220308_PRO.pdf",
"https://static.btgpactual.com/media/prop/42847134000192_20220602_PROP.pdf",
"https://static.btgpactual.com/media/prospecto/42847134000192_20220912_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/42847134000192_20221219_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/42847164000107_20220316_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/42847164000107_20220502_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/42847164000107_20220727_PRO.pdf",
"https://bemdtvm.bradesco/Upload%20Documents/Funds/4938/162/BradescoExplorerPeFipProspectoDefinitivo_Cid_162_2022531842481.pdf",
"https://static.btgpactual.com/media/prospecto/44172951000113_20220131_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/44172951000113_20220517_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/44172951000113_20220715_PRO.PDF",
"https://institucional.xpi.com.br/downloads/BO/1.%20PROT%20VI%20-%20Patria%20Infra%20Energia%20Core%20Renda%20FIP-IE%20-%20Prospecto%20Definitivo.pdf",
"https://www.bancogenial.com/Files/FundosAdm/418_Prospecto_definitivo.pdf",
"https://cloud.luzcapitalmarkets.com.br/.PATRIA_INFRAESTRUTURA_ENERGIA_DEF_CORE_3_EMISSAO_05.pdf",
"https://static.btgpactual.com/media/prospecto/46280082000176_20221220_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/46280082000176_20230525_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/46300253000181_20220830_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/49430776000130_20230210_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/53372547000184_20240125_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/53372547000184_20241025_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/53468219000186_20240116_PRO.pdf",
"https://cloud.luzcapitalmarkets.com.br/.FIP_-_RIZA_AERAS-1_EMISSAO_PROSPECTO_12.pdf",
"https://cloud.luzcapitalmarkets.com.br/.PROSPECTO_PRELIMINAR_COPERNICO_1_EMISSAO_16.pdf",
"https://cloud.luzcapitalmarkets.com.br/.FIP_RIZA_DELPHI_PROSPECTO_DEFINITIVO_1_EMISSAO_15.pdf",
"https://static.btgpactual.com/media/prospecto/55674661000194_20240920_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/56848636000142_20240906_PRO.pdf",
"https://static.btgpactual.com/media/prospecto/56873069000184_20240816_PRO.pdf"
]

resultados = verificar_links(links_testar)
for link in resultados:
    print(link)


In [0]:
import fitz  # PyMuPDF
import os
import openai
import requests

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')
DEPLOYMENT_NAME = "gpt-4.1-mini"

# Caminho com os PDFs curtos
BASE_DIR = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"

def merge_text_blocks(entries):
    merged = []
    buf = []

    def flush_buf():
        if not buf:
            return
        merged.append({
            "type": "text",
            "text": "\n".join(buf).strip()
        })
        buf.clear()

    for e in entries:
        if e["type"] == "text":
            buf.append(e["text"])
        else:
            flush_buf()
            merged.append(e)
    flush_buf()
    return merged

def create_messages_for_pdf(pdf_path):
    results = []
    doc = fitz.open(pdf_path)

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b["type"] == 0:
                lines = b.get("lines", [])
                txt_lines = []
                for line in lines:
                    spans = line.get("spans", [])
                    line_text = "".join(span.get("text", "") for span in spans)
                    if line_text:
                        txt_lines.append(line_text)
                full_text = "\n".join(txt_lines).strip()
                if full_text:
                    results.append({"type": "text", "text": full_text})

    doc.close()
    return merge_text_blocks(results)

def extract_links_from_pdf(pdf_path):
    try:
        messages = [
            {
                "role": "system",
                "content": """
                Você é um assistente que ajuda a identificar links presentes em documentos de texto.

                Sua tarefa é:
                – Analisar o conteúdo textual de um PDF (como um prospecto, regulamento ou aviso).
                – Procurar **todos os links** de documentos do tipo PDF ou DOC (como regulamentos, lâminas, anúncios, planilhas, etc).
                – Os links podem estar em qualquer formato: encurtados, integrais, com ou sem “https”.
                – O retorno deve conter SOMENTE os links (1 por linha), já formatados corretamente (iniciando com http ou https).
                – Se não encontrar nenhum link, retorne apenas a frase: `link não encontrado`.
                """
            },
            {
                "role": "user",
                "content": create_messages_for_pdf(pdf_path)
            }
        ]

        completion = openai.ChatCompletion.create(
            engine=DEPLOYMENT_NAME,
            messages=messages,
            temperature=0.0
        )

        response = completion.choices[0].message['content']
        return response.strip()

    except Exception as e:
        return f"❌ Erro: {str(e)}"

def verificar_link(link):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.head(link, allow_redirects=True, timeout=10, headers=headers)
        content_type = response.headers.get("Content-Type", "").lower()

        if response.status_code < 400 and any(t in content_type for t in ["pdf", "msword", "officedocument.wordprocessingml.document"]):
            return True
    except requests.exceptions.RequestException:
        pass
    return False

def list_all_pdf_paths(base_dir):
    try:
        files = dbutils.fs.ls(base_dir)
        return [
            os.path.join(base_dir, f.name)
            for f in files if f.name.lower().endswith(".pdf")
        ]
    except Exception as e:
        print(f"Erro ao listar PDFs: {str(e)}")
        return []

# 🧩 Etapas principais
pdf_paths = list_all_pdf_paths(BASE_DIR)

dicionario_todos = {}
dicionario_validos = {}

print("🔎 Verificando arquivos...\n")

for pdf_path in pdf_paths:
    nome_arquivo = os.path.basename(pdf_path)
    links_extraidos = extract_links_from_pdf(pdf_path)

    # Divide em múltiplos links e ignora PDFs com mais de um link
    links = [l.strip() for l in links_extraidos.splitlines() if l.startswith("http")]
    if len(links) == 1:
        dicionario_todos[nome_arquivo] = links[0]

        # Verifica validade do link
        if verificar_link(links[0]):
            dicionario_validos[nome_arquivo] = links[0]

# ✅ Resultados
print("🎯 Arquivos com links válidos encontrados:\n")
for nome, link in dicionario_validos.items():
    print(f"{nome}: {link}")



F.I.I._11839593000109_20140122.pdf: http://www.oliveiratrust.com.br/scot/Arquivos/FI-169/20601-1246-20140120101854.pdf

F.I.I._11839593000109_20140508.pdf: http://www.oliveiratrust.com.br/scot/Arquivos/FI-169/20601-1246-20140120101854.pdf

F.I.I._11839593000109_20140805.pdf: http://www.oliveiratrust.com.br/scot/Arquivos/FI-169/20601-1246-20140120101854.pdf

F.I.I._11839593000109_20141022.pdf: http://www.oliveiratrust.com.br/scot/Arquivos/FI-169/20601-1246-20141022092803.pdf

F.I.I._19249956000150_20150624.pdf: http://www.oliveiratrust.com.br/scot/Arquivos/FI-314/53931-2821-20150624180147.pdf

F.I.I._20216935000117_20150602.pdf: http://www.pefran.com.br/empresas2012/banco_bradesco/underwriting/%5B27908%5D-banco_bradesco_underwriting_qfi_prospecto_bradesco_fii/internet/arte/%5B27908%5D-banco_bradesco_underwriting_qfi_Prospecto_bradesco_fii.pdf

FIDC_08692888000182_20140328.pdf: http://www.oliveiratrust.com.br/scot/modulos/downloads/baixar.php?cod=23791

FIDC_08692888000182_20140502.pdf: http://www.oliveiratrust.com.br/scot/modulos/downloads/baixar.php?cod=23791

FIDC_08692888000182_20160712.pdf: http://www.oliveiratrust.com.br/scot/modulos/downloads/baixar.php?cod=97371

FIDC_08692888000182_20200205.pdf: http://www.oliveiratrust.com.br/scot/modulos/downloads/baixar.php?cod=1279191

FIDC_08692888000182_20200810.pdf: http://www.oliveiratrust.com.br/scot/modulos/downloads/baixar.php?cod=1332361

FIDC_09137729000189_20160224.pdf: http://www.petracorretora.com.br/wp-content/uploads/2014/09/Prospecto-SubPreferencial-D_20160224.pdf

FIDC_14166140000149_20180327.pdf: http://corretora.finaxis.com.br/wp-content/uploads/2014/09/prospecto-definitivo-empirica-sifra-star-1.pdf

FIDC_27151223000106_20190619.pdf: http://www.winnerpublicidade.com/pdf/sabemi/180619/Comunicado%20de%20Suspensao%20e%20Prospecto.pdf

FIDC_28279473000199_20171005.pdf: https://bemdtvm.bradesco/Upload%20Documents/Funds/3028/2/Prospecto_Cid_2_2017105165716138.pdf

FIP_17870798000125_20140226.pdf: http://static.gerafuturo.com.br/Documentos/Prospecto_-_P2_Brasil_Infraestrutura_III_FIQFIP.pdf

FIP_30507217000153_20181015.pdf: http://www.vincipartners.com/uploads/VCP%20III%20FIP%20M%20II%20-%20Prospecto%20v.%20limpa.pdf

FIP_33601138000103_20201120.pdf: https://static.btgpactual.com/media/anexo-g1-prospecto-preliminar-versao-diagramada-19112020.pdf

FIP_34218291000100_20201005.pdf: https://static.btgpactual.com/media/fip-ie-perfin-prospecto-definitivo-diagramado.pdf

FIP_35640741000111_20201118.pdf: https://static.btgpactual.com/media/btg-infra-def.pdf

FIP_35640811000131_20200904.pdf: https://static.btgpactual.com/media/fip-econo-real-prospecto-definitivo-31012020.pdf

FIP_35640811000131_20201028.pdf: https://static.btgpactual.com/media/anexo-d1-20201027-btg-fip-economia-real-prospecto-preliminar-diagramado.pdf

FIP_35640811000131_20201214.pdf: https://static.btgpactual.com/media/btg-economia-real-def.pdf

FIP_35640942000119_20201005.pdf: https://static.btgpactual.com/media/1-prot-ix-prisma-proton-fip-ie-prospecto-preliminar.pdf

FIP_35641061000112_20210115.pdf: https://static.btgpactual.com/media/prospecto-definitivo-fip-ie-endurance-debt-11012021.pdf

FIP_36642497000199_20210317.pdf: https://static.btgpactual.com/media/prospecto/36642497000199_20210317_PRO.pdf

FIP_36642570000122_20210425.pdf: https://static.btgpactual.com/media/prospecto/36642570000122_20210419_PRO.pdf

FIP_40011391000164_20210706.pdf: https://static.btgpactual.com/media/prospecto/40011391000164_20210705_PRO.pdf

FIP_40011391000164_20211005.pdf: https://static.btgpactual.com/media/prospecto/40011391000164_20211004_PRO.pdf

FIP_41082947000176_20210721.pdf: https://static.btgpactual.com/media/prospecto/41082947000176_20210721_PRO.pdf

FIP_42120193000164_20211230.pdf: https://static.btgpactual.com/media/prospecto/42120193000164_20211229_PRO.pdf

FIP_42120193000164_20220308.pdf: https://static.btgpactual.com/media/prospecto/42120193000164_20220308_PRO.pdf

FIP_42847134000192_20220602.pdf: https://static.btgpactual.com/media/prop/42847134000192_20220602_PROP.pdf

FIP_42847134000192_20220913.pdf: https://static.btgpactual.com/media/prospecto/42847134000192_20220912_PRO.pdf

FIP_42847134000192_20221219.pdf: https://static.btgpactual.com/media/prospecto/42847134000192_20221219_PRO.pdf

FIP_42847164000107_20220321.pdf: https://static.btgpactual.com/media/prospecto/42847164000107_20220316_PRO.pdf

FIP_42847164000107_20220503.pdf: https://static.btgpactual.com/media/prospecto/42847164000107_20220502_PRO.pdf

FIP_42847164000107_20220728.pdf: https://static.btgpactual.com/media/prospecto/42847164000107_20220727_PRO.pdf

FIP_43391569000138_20220503.pdf: https://bemdtvm.bradesco/Upload%20Documents/Funds/4938/162/BradescoExplorerPeFipProspectoDefinitivo_Cid_162_2022531842481.pdf

FIP_44172951000113_20220208.pdf: https://static.btgpactual.com/media/prospecto/44172951000113_20220131_PRO.pdf

FIP_44172951000113_20220517.pdf: https://static.btgpactual.com/media/prospecto/44172951000113_20220517_PRO.pdf

FIP_44172951000113_20220715.pdf: https://static.btgpactual.com/media/prospecto/44172951000113_20220715_PRO.PDF

FIP_46280082000176_20221221.pdf: https://static.btgpactual.com/media/prospecto/46280082000176_20221220_PRO.pdf

FIP_46280082000176_20230526.pdf: https://static.btgpactual.com/media/prospecto/46280082000176_20230525_PRO.pdf

FIP_46300253000181_20220830.pdf: https://static.btgpactual.com/media/prospecto/46300253000181_20220830_PRO.pdf

FIP_49430776000130_20230213.pdf: https://static.btgpactual.com/media/prospecto/49430776000130_20230210_PRO.pdf

FIP_53372547000184_20240130.pdf: https://static.btgpactual.com/media/prospecto/53372547000184_20240125_PRO.pdf

FIP_53372547000184_20241028.pdf: https://static.btgpactual.com/media/prospecto/53372547000184_20241025_PRO.pdf

FIP_53468219000186_20240123.pdf: https://static.btgpactual.com/media/prospecto/53468219000186_20240116_PRO.pdf

FIP_55067199000167_20240829.pdf: https://cloud.luzcapitalmarkets.com.br/.FIP_-_RIZA_AERAS-1_EMISSAO_PROSPECTO_12.pdf

FIP_55291703000108_20240624.pdf: https://cloud.luzcapitalmarkets.com.br/.PROSPECTO_PRELIMINAR_COPERNICO_1_EMISSAO_16.pdf

FIP_55430054000189_20240625.pdf: https://cloud.luzcapitalmarkets.com.br/.
FIP_RIZA_DELPHI_PROSPECTO_DEFINITIVO_1_EMISSAO_15.pdf

FIP_55674661000194_20240925.pdf: https://static.btgpactual.com/media/prospecto/55674661000194_20240920_PRO.pdf

FIP_56848636000142_20240906.pdf: https://static.btgpactual.com/media/prospecto/56848636000142_20240906_PRO.pdf

FIP_56873069000184_20240820.pdf: https://static.btgpactual.com/media/prospecto/56873069000184_20240816_PRO.pdf


2. Criação da tabela para armazenar os dados da extração:

In [0]:
%sql
DROP TABLE desafio_kinea.prospecto_fundos.extracao_prospectos_kinea

In [0]:
%sql
CREATE TABLE desafio_kinea.prospecto_fundos.extracao_prospectos_kinea (
  nome_fundo STRING,
  valor_cota_emissao STRING,
  direito_preferencia_sobras_montante_adicional STRING,
  taxa_distribuicao_emissao STRING,
  tabela_ativos_fundo STRING,
  sumario_experiencia_socios STRING,
  quantidade_cotas_emissao STRING,
  quantidade_cotas_adicionais_emissao STRING,
  publico_alvo STRING,
  procuracao_AGE STRING,
  planilha_custos STRING,
  ordenar_fatores_risco STRING,
  montante_minimo_emissao STRING,
  investimento_minimo_cpf_cnpj STRING,
  investimento_minimo_inst STRING,
  investimento_maximo_cpf_cnpj STRING,
  investimento_maximo_inst STRING,
  historico_cotacao_bolsa STRING,
  fator_proporcao_dp STRING,
  diluicao_economica_novas_emissoes STRING,
  criterio_rateio STRING,
  carteira_fundos_kinea_intrag STRING,
  breve_historico_gestor STRING,
  percentual_oferta_institucional STRING,
  volume_base_emissao STRING,
  chamada_capital_ipca STRING
)
USING DELTA;



3. Código final para extrair as informações (prompt atualizado):

In [0]:
import fitz
import base64
import re
import os
import openai
import tempfile
import json
from datetime import datetime
from functools import lru_cache
from json import dumps

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

#------------------------------------ LEITURA DO PDF-------------------------------------#
def merge_text_blocks(entries):
    merged = []
    buf = []

    def flush_buf():
        if not buf:
            return
        merged.append({"type": "text", "text": "\n".join(buf).strip()})
        buf.clear()

    for e in entries:
        if e["type"] == "text":
            buf.append(e["text"])
        else:
            flush_buf()
            merged.append(e)
    flush_buf()
    return merged

def create_messages_for_pdf(pdf_path, dpi=200):
    results = []
    doc = fitz.open(pdf_path)
    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b["type"] == 0:
                txt_lines = []
                for line in b.get("lines", []):
                    line_text = "".join(span.get("text", "") for span in line.get("spans", []))
                    if line_text:
                        txt_lines.append(line_text)
                full_text = "\n".join(txt_lines).strip()
                if full_text:
                    results.append({"type": "text", "text": full_text})
            else:
                clip = fitz.Rect(*b["bbox"])
                pix = page.get_pixmap(clip=clip, dpi=dpi)
                img_bytes = pix.tobytes("png")
                b64 = base64.b64encode(img_bytes).decode("ascii")
                results.append({
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{b64}"
                    }
                })
    doc.close()
    return merge_text_blocks(results)

def create_messages(pdf_path):
    return create_messages_for_pdf(pdf_path)



# Função principal para extrair informações usando LLM
def extrai_infos_com_llm(pdf_path):
    prompt = """Role: Act as a highly specialized AI document analyst trained in processing and extracting structured data from official financial documents, specifically related to Investment Funds in Brazil.

    Context:
    You will receive the full text of a regulatory or offering document related to a quota issuance (e.g., prospecto). These documents are usually published in PDF format and contain financial, legal, and operational details about a fund offering.

    Your Objective:
    Extract ** this specific fields** from the document content. Each field corresponds to a relevant piece of information necessary for evaluating the offering. If multiple mentions of the same data point exist, prioritize:
    1. The version labeled as definitive or part of the final offer terms (e.g., \"condições da oferta\").
    2. The most recent or prominent value.
    3. If you are not sure of the information, it's better to return an empty string.
    

    Output Instructions:
    – **Do not infer or fabricate information**. If a field is not explicitly mentioned or lacks clarity, return an **empty string** `\"\"`.
    – All numerical values must use the **international format**: use `.` (dot) for decimals and **no thousand separators**.
    – Return the result as a **strictly valid JSON object**, conforming exactly to the schema below.
    – **Do not include any explanatory text**, headers, comments, or notes outside the JSON object.
    – The order of keys in the output must match the list below.
    – If a field is a list or summary, return a string.

    Informações a extrair (nessa ordem):
        

    Target Output Format:
    Return a single JSON object with the following keys:

    ```json
    {
    \"valor_cota_emissao\": \"\",
    \"direito_preferencia_sobras_montante_adicional\": \"\",
    \"taxa_distribuicao_emissao\": \"\",
    \"tabela_ativos_fundo\": [],
    \"sumario_experiencia_socios\": \"\",
    \"quantidade_cotas_emissao\": \"\",
    \"quantidade_cotas_adicionais_emissao\": \"\",
    \"publico_alvo\": \"\",
    \"procuracao_AGE\": \"\",
    \"planilha_custos\": \"\",
    \"ordenar_fatores_risco\": [],
    \"montante_minimo_emissao\": \"\",
    \"investimento_minimo_cpf_cnpj\": \"\",
    \"investimento_minimo_inst\": \"\",
    \"investimento_maximo_cpf_cnpj\": \"\",
    \"investimento_maximo_inst\": \"\",
    \"historico_cotacao_bolsa\": \"\",
    \"fator_proporcao_dp\": \"\",
    \"diluicao_economica_novas_emissoes\": \"\",
    \"criterio_rateio\": \"\",
    \"carteira_fundos_kinea_intrag\": \"\",
    \"breve_historico_gestor\": \"\",
    \"percentual_oferta_institucional\": \"\",
    \"volume_base_emissao\": \"\",
    \"chamada_capital_ipca\": \"\"
    }
    
    Constraints:
    – Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string.
    – Use Brazilian Portuguese language context for comprehension but ensure numeric formatting adheres to international standards.
    – Be precise and concise. Avoid speculative language.
    

    You are expected to handle varied writing styles and formatting typically found in official financial documents and maintain high accuracy under ambiguity.
    Informações a extrair (nessa ordem):
        1. Valor da Cota da Emissão
        2. Se existe direito de preferência, sobras ou montante adicional
        3. Taxa de distribuição da Emissão
        4. Tabela de ativos do fundo
        5. Sumário de experiência dos sócios
        6. Quantidade de cotas da Emissão
        7. Quantidade de cotas adicionais na Emissão
        8. Público-alvo
        9. Procuração para AGE (se necessária)
        10. Planilha de custos
        11. Ordenação dos fatores de risco
        12. Montante mínimo da Emissão
        13. Investimento mínimo por CPF/CNPJ
        14. Investimento mínimo para investidores institucionais
        15. Investimento máximo por CPF/CNPJ
        16. Investimento máximo para investidores institucionais
        17. Histórico de cotação em bolsa (mínima, média e máxima)
        18. Fator de proporção para o Direito de Preferência (DP)
        19. Diluição econômica em novas emissões
        20. Critério de rateio
        21. Carteira de fundos Kinea com Intrag
        22. Breve histórico do gestor
        23. % da oferta destinada ao público institucional
        24. Volume base da Emissão
    """  

    # Extrai o conteúdo do PDF como texto simples
    doc = fitz.open(pdf_path)
    texto_parcial = ""
    for i in range(start_page, end_page_exclusive):
        texto_parcial += doc[i].get_text() + "\n"
    doc.close()

    message_text = [
        {'role': 'system', 'content': prompt},
        {"role": "user", "content": texto_parcial}
    ]

    func = {
        'name': 'extrair_informacao_pdf',
        'description': 'Função para extrair os valores do PDF.',
        'parameters': {
            "type": "object",
            "properties": {
                "valor_cota_emissao": {
                "type": "string",
                "description": """Preço unitário de subscrição por cota na nova emissão. Informe como número (ex.: \"98.45\") ou texto vazio se não disponível.
                Consultar texto introdutório do prospecto. Informação pode estar no formato: "...nominativas e escriturais, da 4ª Emissão ("4ª Emissão" e "Novas Cotas"), pelo valor unitário de R$ 101,26 (cento e um reais e vinte e seis centavos), correspondente ao valor patrimonial das cotas da Classe... Caso não exista, retornar string vazia."""
                },
                "direito_preferencia_sobras_montante_adicional": {
                "type": "string",
                "description": """Indique se há ou não direito de preferência, possibilidade de sobras ou de montante adicional. Ex.: \"Sim\" ou \"Não\".
                A informação pode estar na seção com nome "Informações sobre a existência de direito de preferência na subscrição de novas cotas" ou similar. Caso não exista, retornar string vazia."""
                },
                "taxa_distribuicao_emissao": {
                "type": "string",
                "description": "Percentual ou valor fixo da taxa de distribuição primária (remuneração do coordenador líder). Ex.: \"3.0%\". Consultar texto introdutório do prospecto. A informação pode estar numa tabela seguida do número por extenso. Ex: 2,00% (dois por cento) . Caso não exista, retornar string vazia."
                },
                "tabela_ativos_fundo": {
                "type": "string",
                "description": """Cópia ou referência resumida da tabela/lista dos principais ativos que compõem a carteira-alvo ou carteira atual do fundo.Liste os ativos somente com os nomes deles (seguidos da sigla se tiver ex: CRI) separados por vírgula. Não é necessário numerar. A informação pode estar na seção 'Destinação dos Recursos' ou similar. A informação pode estar no formato '...este sentido, os Ativos nos quais o Fundo poderá investir são: (i) Certificados de Recebíveis Imobiliários ("CRI"); (ii) cotas de fundos de investimento imobiliário ("FII"); (iii) debêntures ("Debêntures") emitidas por emissores devidamente autorizados nos termos da regulamentação aplicável, e cujas atividades preponderantes sejam permitidas aos FII... Caso não exista, retornar string vazia."""
                },
                "sumario_experiencia_socios": {
                "type": "string",
                "description": "Síntese da experiência profissional dos sócios/gestores apresentada no documento. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_emissao": {
                "type": "string",
                "description": "Quantidade total de cotas ofertadas na emissão base (sem considerar lotes adicionais). Ex.: \"5.000.000\". Consultar texto inicial do prospecto. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_adicionais_emissao": {
                "type": "string",
                "description": "Quantidade de cotas do lote adicional ou suplementar, se previsto. Consultar texto inicial do prospecto. A informação pode estar no formato '... Os Ofertantes, nos termos e conforme os limites estabelecidos no artigo 50 da Resolução CVM nº 160, com a prévia concordância do Coordenador Líder (conforme abaixo definido), poderão optar por distribuir um volume adicional de até 25% (vinte e cinco por cento) da quantidade máxima de Novas Cotas inicialmente ofertadas, ou seja, até 1.234.445 (um milhão, duzentas e trinta e quatro mil, quatrocentas e quarenta e cinco) Novas Cotas...' . Caso não exista, retornar string vazia."
                },
                "publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta (ex.: \"Investidores em geral\", \"Investidores profissionais\", etc.). A informação pode estar na seção 'Identificação do público alvo' ou similar. Caso não exista, retornar string vazia."
                },
                "procuracao_AGE": {
                "type": "string",
                "description": "Indicação se o documento contém modelo de procuração para AGE ou se não é necessária. A informação pode estar na seção 'Indicar a eventual possibilidade de destinação dos recursos a quaisquer ativos em relação às quais possa haver conflito de interesse, informando as aprovações necessárias existentes e/ou a serem obtidas, incluindo nesse caso nos fatores de risco, explicação objetiva sobre a falta de transparência na formação dos preços destas operações' ou similar. Ex.: \"Modelo de procuração incluso\" ou \"Não aplicável\".Caso não exista, retornar string vazia."
                },
                "planilha_custos": {
                "type": "string",
                "description": "Referência ou resumo da planilha de custos da oferta (taxas, despesas, previsão de custos). A informação pode estar na seção 'Demonstrativo do custo da distribuição, discriminando: a) a porcentagem em relação ao preço unitário de subscrição; b) a comissão de coordenação; c) a comissão de distribuição; d) a comissão de garantia de subscrição, se houver; e) outras comissões (especificar); f) os tributos incidentes sobre as comissões, caso estes sejam arcados pela classe de cotas; g) o custo unitário de distribuição; h) as despesas decorrentes do registro de distribuição; e i) outros custos relacionados' ou similar. Ex: \"Taxa de distribuição total 2.00%, comissão de coordenação 0.10%, comissão de distribuição 1.70%, advogados 0.04%, tributos advogados 0.00%, taxa registro CVM 0.04%, taxa registro Anbima 0.00%, taxa B3 distribuição padrão fixa 0.03%, taxa B3 distribuição padrão variável 0.04%, taxa B3 análise oferta 0.00%, taxa B3 listagem 0.00%, anúncio 0.00%, cartório 0.00%, outras despesas 0.02%.\"Caso não exista, retornar string vazia. "
                },
                "ordenar_fatores_risco": {
                "type": "string",
                "description": "Fatores de risco já em ordem apresentada ou indicação de onde ela se encontra. Indicar o nome do risco seguido da sua intensidade ex: Risco de Crédito (Maior). Não use listas, somente strings simples.A informação pode estar na seção 'Fatores de Risco' ou similar.Caso não exista, retornar string vazia."
                },
                "montante_minimo_emissao": {
                "type": "string",
                "description": "Valor mínimo da oferta para que a emissão seja considerada válida (montante mínimo de liquidação). A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_cpf_cnpj": {
                "type": "string",
                "description": "Valor mínimo de subscrição por investidor pessoa física ou jurídica (CPF/CNPJ).A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_inst": {
                "type": "string",
                "description": "Valor mínimo de subscrição exigido para investidores institucionais, se diferente. A informação pode estar na seção  'Identificação do público-alvo' ou similar. Consulte para saber se investidores institucionais são contemplados. Se sim, extrair o valor mínimo de investimento para esse caso. Caso não exista, retornar string vazia."
                },
                "investimento_maximo_cpf_cnpj": {
                "type": "string",
                "description": "Valor máximo permitidos por investidor pessoa física ou jurídica (CPF/CNPJ), se houver restrição. A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_maximo_inst": {
                "type": "string",
                "description": "Valor máximo permitido para investidores institucionais, se houver. A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "historico_cotacao_bolsa": {
                "type": "string",
                "description": "Valores mínima, média e máxima da cota em bolsa conforme apresentado (ex.: \"Mín 90.10 / Méd 94.50 / Máx 98.00\").A informação pode estar na seção 'Cotação em bolsa de valores ou mercado de balcão dos valore mobiliários a serem distribuídos, inclusive no exterior' ou similar.Caso não exista, retornar string vazia."
                },
                "fator_proporcao_dp": {
                "type": "string",
                "description": "Fator de proporção do Direito de Preferência (DP). A informação pode estar na seção 'Informações sobre a existência de direito de preferência na subscrição de novas cotas'. Extraia a proporção, se existir, no mesmo formato descrito no documento.Caso não exista, retornar string vazia."
                },
                "diluicao_economica_novas_emissoes": {
                "type": "string",
                "description": "Percentual estimado de diluição econômica para cotistas atuais após a nova emissão, ou texto explicativo.Caso não exista, retornar string vazia."
                },
                "criterio_rateio": {
                "type": "string",
                "description": "Critério de rateio em caso de excesso de demanda. A informação pode estar na seção 'Critério de Rateio' ou similar. Ex.: \"Pro rata\" ou \"Prioridade investidores profissionais\".Caso não exista, retornar string vazia."
                },
                "carteira_fundos_kinea_intrag": {
                "type": "string",
                "description": "Informação específica sobre a carteira de fundos Kinea com Intrag, se mencionada.Caso não exista, retornar string vazia."
                },
                "breve_historico_gestor": {
                "type": "string",
                "description": "Resumo do histórico da gestora ou do gestor responsável conforme descrito no documento.Caso não exista, retornar string vazia."
                },
                "percentual_oferta_institucional": {
                "type": "string",
                "description": "Percentual da oferta reservado ao público institucional. Ex.: \"30%\".Caso não exista, retornar string vazia."
                },
                "volume_base_emissao": {
                "type": "string",
                "description": "Montante financeiro (R$) correspondente ao volume base da emissão sem considerar lote adicional.Caso não exista, retornar string vazia."
                },
                "chamada_capital_ipca": {
                "type": "string",
                "description": "Descrição de cláusula de chamada de capital com atualização pelo IPCA, se existente. Caso não exista, retornar string vazia."
                }
            },
            "required": [
                "valor_cota_emissao",
                "direito_preferencia_sobras_montante_adicional",
                "taxa_distribuicao_emissao",
                "tabela_ativos_fundo",
                "sumario_experiencia_socios",
                "quantidade_cotas_emissao",
                "quantidade_cotas_adicionais_emissao",
                "publico_alvo",
                "procuracao_AGE",
                "planilha_custos",
                "ordenar_fatores_risco",
                "montante_minimo_emissao",
                "investimento_minimo_cpf_cnpj",
                "investimento_minimo_inst",
                "investimento_maximo_cpf_cnpj",
                "investimento_maximo_inst",
                "historico_cotacao_bolsa",
                "fator_proporcao_dp",
                "diluicao_economica_novas_emissoes",
                "criterio_rateio",
                "carteira_fundos_kinea_intrag",
                "breve_historico_gestor",
                "percentual_oferta_institucional",
                "volume_base_emissao",
                "chamada_capital_ipca"
            ]
        }
    }

    try:
        completion = openai.ChatCompletion.create(
            engine="gpt-4.1-mini",  # Corrigido o nome do engine
            messages=message_text,
            temperature=0.1,
            top_p=0.95,
            tools=[{"type": "function", "function": func}],
            tool_choice={"type": "function", "function": {"name": func["name"]}}
        )

        responseText = completion.to_dict()['choices'][0]['message']
        arguments_str = responseText['tool_calls'][0]["function"]["arguments"]
        
        # Usar json.loads ao invés de eval para segurança
        return json.loads(arguments_str)
        
    except Exception as e:
        print(f"Erro na chamada da LLM: {e}")
        return {}

# Caminho do volume Databricks
caminho_volume = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"

# Lista manual de PDFs para processar
arquivos_pdf_especificos = [
    #"/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/FIDC_60431592000128_20250514.pdf",
    "/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/FIDC_60431592000128_20250623.pdf",
    "/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/FII_42754362000118_20250617.pdf",
    "/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/FII_30130708000128_20250617.pdf"
    # Adicione outros caminhos aqui conforme necessário
    
]

# Campos esperados na tabela
campos_esperados = [
    "nome_fundo",
    "valor_cota_emissao",
    "direito_preferencia_sobras_montante_adicional",
    "taxa_distribuicao_emissao",
    "tabela_ativos_fundo",
    "sumario_experiencia_socios",
    "quantidade_cotas_emissao",
    "quantidade_cotas_adicionais_emissao",
    "publico_alvo",
    "procuracao_AGE",
    "planilha_custos",
    "ordenar_fatores_risco",
    "montante_minimo_emissao",
    "investimento_minimo_cpf_cnpj",
    "investimento_minimo_inst",
    "investimento_maximo_cpf_cnpj",
    "investimento_maximo_inst",
    "historico_cotacao_bolsa",
    "fator_proporcao_dp",
    "diluicao_economica_novas_emissoes",
    "criterio_rateio",
    "carteira_fundos_kinea_intrag",
    "breve_historico_gestor",
    "percentual_oferta_institucional",
    "volume_base_emissao",
    "chamada_capital_ipca",
]

# Loop somente nos PDFs da lista fornecida
for caminho_pdf in arquivos_pdf_especificos:
    try:
        nome_fundo = os.path.basename(caminho_pdf).replace(".pdf", "")
        print(f"🔄 Processando: {nome_fundo}")
        
        # Usar a função correta com LLM
        dados_extraidos = extrai_infos_com_llm(caminho_pdf)
        dados_extraidos["nome_fundo"] = nome_fundo

        # Mapeamento dos campos extraídos para os nomes corretos da tabela
        mapeamento_campos = {
            "direito_preferencia_sobras_montante": "direito_preferencia_sobras_montante_adicional",
            "quantidade_cotas_adicionais": "quantidade_cotas_adicionais_emissao",
            "montante_minimo": "montante_minimo_emissao",
            "taxa_distribuicao": "taxa_distribuicao_emissao",
            "investimento_minimo": "investimento_minimo_cpf_cnpj",
            "investimento_maximo": "investimento_maximo_cpf_cnpj",
            "historico_cotacao": "historico_cotacao_bolsa",
            "fator_proporcao": "fator_proporcao_dp",
            "diluicao_economica": "diluicao_economica_novas_emissoes",
            "percentual_institucional": "percentual_oferta_institucional",
            "volume_base": "volume_base_emissao",
            "carteira_kinea_intrag": "carteira_fundos_kinea_intrag"
        }

        # Converter listas para strings e aplicar mapeamento de nomes
        dados_convertidos = {}
        for k, v in dados_extraidos.items():
            # Aplicar mapeamento de nomes se necessário
            nome_campo = mapeamento_campos.get(k, k)
            
            if isinstance(v, list):
                dados_convertidos[nome_campo] = ", ".join(str(item) for item in v)
            else:
                dados_convertidos[nome_campo] = str(v) if v is not None else ""

        # Garantir que todos os campos esperados existam
        for campo in campos_esperados:
            if campo not in dados_convertidos:
                dados_convertidos[campo] = ""

        # Log para debug
        print(f"📊 Dados extraídos para {nome_fundo}:")
        for k, v in dados_convertidos.items():
            if v.strip():  # Só mostra campos que não estão vazios
                print(f"  {k}: {v[:100]}...")  # Mostra só os primeiros 100 chars

        
        # Remove quaisquer campos não esperados (como 'taxa_distribuicao')
        dados_convertidos = {k: v for k, v in dados_convertidos.items() if k in campos_esperados}

        df_novo = spark.createDataFrame([dados_convertidos])
        df_novo.write.format("delta").mode("append").saveAsTable("desafio_kinea.prospecto_fundos.extracao_prospectos_kinea")

        print(f"✅ Inserido com sucesso: {nome_fundo}")

    except Exception as e:
        print(f"❌ Erro ao processar {caminho_pdf}: {e}")
        import traceback
        traceback.print_exc()

 **4**. Criando tabela de extração de dados com relação aos prospectos não Kinea

In [0]:
%sql
CREATE TABLE desafio_kinea.prospecto_fundos.extracao_prospectos_v3 (
  tipo STRING,
  cnpj STRING,
  data_emissao STRING,
  qt_emissoes STRING,
  nome_fundo STRING,
  valor_cota_emissao STRING,
  direito_preferencia_sobras_montante_adicional STRING,
  taxa_distribuicao_emissao STRING,
  tabela_ativos_fundo STRING,
  sumario_experiencia_socios STRING,
  quantidade_cotas_emissao STRING,
  quantidade_cotas_adicionais_emissao STRING,
  publico_alvo STRING,
  obs_publico_alvo STRING,
  procuracao_AGE STRING,
  planilha_custos STRING,
  ordenar_fatores_risco STRING,  -- Campo ajustado para STRING
  montante_minimo_emissao STRING,
  investimento_minimo_cpf_cnpj STRING,
  investimento_minimo_inst STRING,
  investimento_maximo_cpf_cnpj STRING,
  investimento_maximo_inst STRING,
  historico_cotacao_bolsa STRING,
  fator_proporcao_dp STRING,
  diluicao_economica_novas_emissoes STRING,
  criterio_rateio STRING,
  carteira_fundos_kinea_intrag STRING,
  breve_historico_gestor STRING,
  percentual_oferta_institucional STRING,
  volume_base_emissao STRING,
  chamada_capital_ipca STRING
)
USING DELTA;



In [0]:
%sql
Drop TABLE desafio_kinea.prospecto_fundos.extracao_prospectos_v3

5. Leitura de prospectos outras empresas

In [0]:
#Codigo Final para prospectos
import fitz
import pdfplumber 
import base64
import re
import os
import openai
import tempfile
import json
import io
from PIL import Image
from datetime import datetime
from functools import lru_cache
from json import dumps
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

#Função para detectar se pdf esta em formato coluna
def is_columnar_layout(caminho_pdf,  threshold_gap=50):
    """Detecta se o PDF tem layout em colunas analisando a segunda página, pois a primeira
    normalmente não tem layout em colunas"""
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            if len(pdf.pages) < 4:
                pagina = pdf.pages[1]  # segunda página
                words = pagina.extract_words()
            else:
                pagina = pdf.pages[15]  # desde a 16 página
                words = pagina.extract_words()

            if not words:
                return False

            # Ordena as palavras por coordenada x0
            words_sorted = sorted(words, key=lambda w: w['x0'])

            # Agrupa as palavras em clusters horizontais baseado na distância
            clusters = []
            current_cluster = [words_sorted[0]]
            
            for w in words_sorted[1:]:
                prev = current_cluster[-1]
                gap = w['x0'] - prev['x1']  # distância entre fim da palavra anterior e início da atual

                if gap > threshold_gap:
                    # Começa um novo cluster
                    clusters.append(current_cluster)
                    current_cluster = [w]
                else:
                    current_cluster.append(w)
            clusters.append(current_cluster)

            # Se houver 2 clusters separados por um gap muito grnade, provavelmente esta em formato coluna
            if len(clusters) < 2:
                return False

            # Calcula distância horizontal entre clusters
            distances = []
            for i in range(len(clusters) - 1):
                max_x1 = max(w['x1'] for w in clusters[i])
                min_x0 = min(w['x0'] for w in clusters[i+1])
                distances.append(min_x0 - max_x1)

            # Se pelo menos um gap for maior que threshold, considera colunas
            return any(d > threshold_gap for d in distances)

    except Exception as e:
        print(f"Erro ao detectar layout: {e}")
        return False

#Função para extrair texto de pdfs em formato coluna
def extrair_texto_colunas_pdf(caminho_pdf):
    """Extrai texto de PDFs com layout em colunas usando pdfplumber"""
    texto_total = ""
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            largura = pagina.width
            altura = pagina.height
            # Extrai coluna esquerda
            col_esq = pagina.crop((0, 0, largura/2, altura)).extract_text() or ""
            # Extrai coluna direita
            col_dir = pagina.crop((largura/2, 0, largura, altura)).extract_text() or ""
            texto_total += col_esq + "\n" + col_dir + "\n\n"
    return texto_total.replace("\t", "").strip()


def extrair_texto_pdf(caminho_pdf):
    if is_columnar_layout(caminho_pdf):
        print("📄 Detectado layout em colunas - usando pdfplumber")
        return extrair_texto_colunas_pdf(caminho_pdf)
    
    print("📄 Layout normal - usando fitz")
    return "\n".join(dividir_pdf_em_duas_partes(caminho_pdf))

def dividir_pdf_em_duas_partes(caminho_pdf):
    """Divide o PDF em duas partes pelo número de páginas"""
    doc = fitz.open(caminho_pdf)
    total_paginas = len(doc)
    metade = total_paginas // 2

    texto_parte1 = ""
    texto_parte2 = ""

    for i in range(total_paginas):
        texto = doc[i].get_text()
        if i < metade:
            texto_parte1 += texto
        else:
            texto_parte2 += texto

    doc.close()
    return texto_parte1.strip(), texto_parte2.strip()


# A função a seguir:
# - recebe um texto (no caso o PDF convertido)
# - retorna um dicionário com as informações extraídas.

def extrai_infos_prospectos(texto):
    prompt = f"""Role: Act as a highly specialized AI document analyst trained in processing and extracting structured data from official financial documents, specifically related to Investment Funds in Brazil.

    Context:
    You will receive the full text of a regulatory or offering document related to a quota issuance (e.g., prospecto). These documents are usually published in PDF format and contain financial, legal, and operational details about a fund offering.

    Your Objective:
    Extract ** this specific fields** from the document content. Each field corresponds to a relevant piece of information necessary for evaluating the offering. If multiple mentions of the same data point exist, prioritize:
    1. The version labeled as definitive or part of the final offer terms (e.g., \"condições da oferta\").
    2. The most recent or prominent value.
    3. If you are not sure of the information, it's better to return an empty string.
    

    Output Instructions:
    – **Do not infer or fabricate information**. If a field is not explicitly mentioned or lacks clarity, return an **empty string** \"\".
    – All numerical values must use the **international format**: use . (dot) for decimals and **no thousand separators**.
    – Return the result as a **strictly valid JSON object**, conforming exactly to the schema below.
    – **Do not include any explanatory text**, headers, comments, or notes outside the JSON object.
    – The order of keys in the output must match the list below.
    – If a field is a list or summary, return a string.

    
    Constraints:
    – Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string.
    – Use Brazilian Portuguese language context for comprehension but ensure numeric formatting adheres to international standards.
    – Be precise and concise. Avoid speculative language.
    

    You are expected to handle varied writing styles and formatting typically found in official financial documents and maintain high accuracy under ambiguity.
    Informações a extrair (nessa ordem):
        1. CNPJ do fundo
        2. Data de emissão do fundos
        3. Quantidade de emissões que aquele fundo já passou
        4. Nome do Fundo
        5. Valor da Cota da Emissão
        6. Se existe direito de preferência, sobras ou montante adicional
        7. Taxa de distribuição da Emissão
        8. Tabela de ativos do fundo
        9. Sumário de experiência dos sócios
        10. Quantidade de cotas da Emissão
        11. Quantidade de cotas adicionais na Emissão
        12. Público-alvo
        13. Procuração para AGE (se necessária)
        14. Planilha de custos
        15. Ordenação dos fatores de risco
        16. Montante mínimo da Emissão
        17. Investimento mínimo por CPF/CNPJ
        18. Investimento mínimo para investidores institucionais
        19. Investimento máximo por CPF/CNPJ
        20. Investimento máximo para investidores institucionais
        21. Histórico de cotação em bolsa (mínima, média e máxima)
        22. Fator de proporção para o Direito de Preferência (DP)
        23. Diluição econômica em novas emissões
        24. Critério de rateio
        25. Carteira de fundos Kinea com Intrag
        26. Breve histórico do gestor
        27. % da oferta destinada ao público institucional
        28. Volume base da Emissão
    """
    message_text = [
        {'role': 'system', 'content': prompt},
        {"role": "user", "content": texto}
    ]

    func = {
        'name': 'extrair_informacao_pdf',
        'description': 'Função para extrair os valores do PDF.',
        'parameters': {
            "type": "object",
            "properties": {
                "qt_emissoes": {
                "type": "string",
                "description": "Quantidade de emissões que aquele fundo já passou. Informe como número (ex.: \"1\", \"2\", etc.). A informação pode estar no formato 'PROSPECTO DEFINITIVO DE DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 1ª (PRIMEIRA) EMISSÃO...' ou então 'DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 4ª (QUARTA) EMISSÃO...'." 
                },
                "nome_fundo": {
                "type": "string",
                "description": "Extrair o nome do fundo"
                },
                "valor_cota_emissao": {
                "type": "string",
                "description": """Preço/valor unitário de subscrição por cota na nova emissão. Informe como número (ex.: \"98.45\") ou texto vazio se não disponível.
                Consultar texto introdutório do prospecto mas pode estar em outras partes do texto, pode ser encontrado como valor unitário inicial. Informação pode estar no formato: "...Serão emitidas inicialmente cotas no valor de R$10,00 (dez reais) cada, independentemente da classe..." ou como "valor unitário inicial na data da primeira subscrição de R$1,00 (um real)...". Caso não exista, retornar string vazia."""
                },
                "direito_preferencia_sobras_montante_adicional": {
                "type": "string",
                "description": """Indique se os investidores têm direito ou não de preferência para subscrição de sobras ou lote/montante adicional de cotas. Ex.: \"Sim\" ou \"Não\".
                A informação pode estar na seção "Distribuição Parcial", “Lote Adicional”, “Outras Características da Oferta”, “Montante Adicional” ou similar. Caso não exista, retornar string vazia."""
                },
                "taxa_distribuicao_emissao": {
                "type": "string",
                "description": "Percentual ou valor fixo da taxa de distribuição primária (remuneração do coordenador líder). Ex.: \"3.0%\". A informação pode estar numa tabela seguida do número por extenso. Extraia o valor do percentual em relação ao valor da cota ou então o percentual em relação ao montante total. Ex: 2,00% (dois por cento) . Caso não exista, retornar string vazia."
                },
                "tabela_ativos_fundo": {
                "type": "string",
                "description": """Cópia ou referência resumida da tabela/lista dos principais ativos que compõem a carteira-alvo ou carteira atual do fundo. Liste os ativos somente com os nomes deles (seguidos da sigla se tiver ex: CRI) separados por vírgula, não necessita da explicação sobre o ativo caso encontre. Não é necessário numerar. A informação pode estar na seção 'Destinação dos Recursos', 'Política de Investimentos', 'Composição da Carteira', 'Objetivo do Fundo' ou similar. A informação pode estar no formato '...este sentido, os Ativos nos quais o Fundo poderá investir são: (i) Certificados de Recebíveis Imobiliários ("CRI"); (ii) cotas de fundos de investimento imobiliário ("FII"); (iii) debêntures ("Debêntures") emitidas por emissores devidamente autorizados nos termos da regulamentação aplicável, e cujas atividades preponderantes sejam permitidas aos FII... Caso não exista, retornar string vazia."""
                },
                "sumario_experiencia_socios": {
                "type": "string",
                "description": "Síntese da experiência profissional dos sócios/gestores apresentada no documento. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_emissao": {
                "type": "string",
                "description": "Quantidade total de cotas ofertadas na emissão base (sem considerar lotes adicionais). Ex.: \"5.000.000\". Consultar texto inicial do prospecto. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_adicionais_emissao": {
                "type": "string",
                "description": "Quantidade de cotas do lote adicional ou suplementar, se previsto. Consultar texto inicial do prospecto. A informação pode estar no formato '...a quantidade de Cotas inicialmente ofertada poderá ser acrescida em até 25% (vinte e cinco por cento), ou seja, em até 7.500.000 (sete milhões e quinhentas mil) Cotas...' . Caso não exista, retornar string vazia."
                },
                "publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta (ex.: \"Investidores em geral\", \"Investidores profissionais\", etc.). A informação pode estar na seção 'Identificação do público alvo' ou similar. Caso não exista, pode retornar como 'Público Geral'."
                },
                "procuracao_AGE": {
                "type": "string",
                "description": "Indicação se o documento contém modelo de procuração para AGE ou se não é necessária. A informação pode estar geralmente no final do prospecto ou em seção chamada 'Procuração', 'Assembleia Geral', 'Anexos1' ou similar. Ex.: \"Modelo de procuração incluso\" ou \"Não aplicável\".Caso não exista, retornar string vazia."
                },
                "planilha_custos": {
                "type": "string",
                "description": "Referência ou resumo da planilha de custos da oferta (incluindo taxas, despesas, previsão de custos). A informação pode estar na seção  'Custos da Distribuição', 'Demonstrativo de Custos', 'Contrato de Distribuição'. discriminando: a) a porcentagem em relação ao preço unitário de subscrição; b) a comissão de coordenação; c) a comissão de distribuição; d) a comissão de garantia de subscrição, se houver; e) outras comissões (especificar); f) os tributos incidentes sobre as comissões, caso estes sejam arcados pela classe de cotas; g) o custo unitário de distribuição; h) as despesas decorrentes do registro de distribuição; e i) outros custos relacionados ou similar. Ex: \"a) comissão de distribuição: 2.0%... b) comissão de coordenação: 0.5%...\". Caso não exista, retornar string vazia. "
                },
                "ordenar_fatores_risco": {
                "type": "string",
                "description": "Fatores de risco já em ordem apresentada ou indicação de onde ela se encontra. Indicar o nome do risco seguido da sua intensidade ex: Risco de Crédito (Maior); não necessita da explicação sobre o risco caso encontre. Procure por palavras chaves como 'créditos' e 'derivativos'. Não use listas, somente strings simples. A informação pode estar na seção 'Fatores de Risco' ou similar. Caso não exista, retornar string vazia."
                },
                "montante_minimo_emissao": {
                "type": "string",
                "description": "Valor mínimo da oferta para que a emissão seja considerada válida (montante mínimo de liquidação). A informação pode estar na seção 'Distribuição Parcial', 'Montante Mínimo', 'Características da Oferta' ou similar. Pode estar acompanhado do lote mínimo, Ex: 'observado o lote mínimo, de 10 (dez) Quotas... correspondentes ao valor de investimento de R$2.000,00 (Dois mil reais)'.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_cpf_cnpj": {
                "type": "string",
                "description": "Valor mínimo de subscrição por investidor pessoa física ou jurídica (CPF/CNPJ). A informação pode estar na seção 'Aplicação Mínima', 'Investimento Mínimo', 'Público Alvo' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_inst": {
                "type": "string",
                "description": "Valor mínimo de subscrição exigido para investidores institucionais, se diferente. A informação pode estar na seção  'Público-alvo', 'Distribuição para Institucionais' ou similar. Consulte para saber se investidores institucionais são contemplados. Se sim, extrair o valor mínimo de investimento para esse caso. Caso não exista, retornar string vazia."
                },
                "investimento_maximo_cpf_cnpj": {
                "type": "string",
                "description": "Valor máximo permitidos por investidor pessoa física ou jurídica (CPF/CNPJ), se houver restrição. A informação pode estar na seções 'Público-Alvo' ou cláusulas com restrições de alocação ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_maximo_inst": {
                "type": "string",
                "description": "Valor máximo permitido para investidores institucionais, se houver. A informação pode estar na seção  'Público-alvo', 'Distribuição para Institucionais' ou similar.Caso não exista, retornar string vazia."
                },
                "historico_cotacao_bolsa": {
                "type": "string",
                "description": "Valores mínima, média e máxima da cota em bolsa conforme apresentado (ex.: \"Mín 90.10 / Méd 94.50 / Máx 98.00\").A informação pode estar na seção 'Cotação em Bolsa', 'Informações sobres as cotas' ou similar.Caso não exista, retornar string vazia."
                },
                "fator_proporcao_dp": {
                "type": "string",
                "description": "Fator de proporção do Direito de Preferência (DP). A informação pode estar na seção 'Distribuição Parcial', 'Lote Adicional', 'Rateio de Cotas', ou na parte que trata do 'Plano de Distribuição'. Extraia a proporção, se existir, no mesmo formato descrito no documento.Caso não exista, retornar string vazia."
                },
                "diluicao_economica_novas_emissoes": {
                "type": "string",
                "description": "Percentual estimado de diluição econômica para cotistas atuais após a nova emissão, ou texto explicativo.Caso não exista, retornar string vazia."
                },
                "criterio_rateio": {
                "type": "string",
                "description": "Critério de rateio em caso de excesso de demanda. A informação pode estar na seção 'Critério de Rateio', 'Outras Características da Oferta' ou similar. Ex.: \"Pro rata\" ou \"Prioridade investidores profissionais\".Caso não exista, retornar string vazia."
                },
                "percentual_oferta_institucional": {
                "type": "string",
                "description": "Percentual da oferta reservado ao público institucional. Ex.: \"30%\".Caso não exista, retornar string vazia."
                },
                "volume_base_emissao": {
                "type": "string",
                "description": "Montante financeiro (R$) correspondente ao volume base da emissão sem considerar lote adicional.Caso não exista, retornar string vazia."
                },
                "chamada_capital_ipca": {
                "type": "string",
                "description": "Descrição de cláusula de chamada de capital com atualização pelo IPCA, se existente. Caso não exista, retornar string vazia."
                }
            },
            "required": [
                #"cnpj",
                #"data_emissao",
                "qt_emissoes",
                "nome_fundo",
                "valor_cota_emissao",
                "direito_preferencia_sobras_montante_adicional",
                "taxa_distribuicao_emissao",
                "tabela_ativos_fundo",
                "sumario_experiencia_socios",
                "quantidade_cotas_emissao",
                "quantidade_cotas_adicionais_emissao",
                "publico_alvo",
                "obs_publico_alvo",
                "procuracao_AGE",
                "planilha_custos",
                "ordenar_fatores_risco",
                "montante_minimo_emissao",
                "investimento_minimo_cpf_cnpj",
                "investimento_minimo_inst",
                "investimento_maximo_cpf_cnpj",
                "investimento_maximo_inst",
                "historico_cotacao_bolsa",
                "fator_proporcao_dp",
                "diluicao_economica_novas_emissoes",
                "criterio_rateio",
                "carteira_fundos_kinea_intrag",
                "breve_historico_gestor",
                "percentual_oferta_institucional",
                "volume_base_emissao",
                "chamada_capital_ipca"
            ]
        }
    }

    try:
        completion = openai.ChatCompletion.create(
            engine="gpt-4.1-mini",  
            messages=message_text,
            temperature=0.1,
            top_p=0.95,
            tools=[{"type": "function", "function": func}],
            tool_choice={"type": "function", "function": {"name": func["name"]}}
        )

        responseText = completion.to_dict()['choices'][0]['message']
        arguments_str = responseText['tool_calls'][0]["function"]["arguments"]
        
        # Parse do JSON retornado pela função
        dados_extraidos = json.loads(arguments_str)
        
        return dados_extraidos
        
    except json.JSONDecodeError as e:
        print(f"Erro ao fazer parse do JSON: {e}")
        return {"erro": "Falha ao processar resposta da IA - JSON inválido"}
        
    except KeyError as e:
        print(f"Erro de estrutura na resposta: {e}")
        return {"erro": "Falha na estrutura da resposta da IA"}
        
    except Exception as e:
        print(f"Erro geral: {e}")
        return {"erro": f"Erro inesperado: {str(e)}"}
            

#extraindo cnpj, tipo de arquivo e data de emissão
def extrair_info_arquivo(nome_arquivo):
    """
    Extrai o CNPJ e a data (YYYY-MM-DD) do nome do arquivo.
    Exemplo de entrada: 'FIDC_06182371000118_20080130.pdf'
    """
    nome_base = os.path.basename(nome_arquivo)
    match = re.match(r"^(F\.I\.I\.|FIDC|FIP)_(\d{14})_(\d{8})\.pdf$", nome_base)

    if match:
        tipo = match.group(1)
        cnpj_raw = match.group(2)
        data_raw = match.group(3)

        cnpj_formatado = f"{cnpj_raw[:2]}.{cnpj_raw[2:5]}.{cnpj_raw[5:8]}/{cnpj_raw[8:12]}-{cnpj_raw[12:]}"
        data_formatada = f"{data_raw[:4]}-{data_raw[4:6]}-{data_raw[6:]}"
        return tipo, cnpj_formatado, data_formatada
    else:
        return "", ""
    

def main():
    # Onde nossos pdf estão
    diretorio_pdfs = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf"
    
    # Listando os PDFs encontrados no diretorio
    lista_pdfs = [
        os.path.join(diretorio_pdfs, arquivo) 
        for arquivo in os.listdir(diretorio_pdfs) 
        if arquivo.lower().endswith(".pdf") and 
           any(prefixo in arquivo.upper() for prefixo in ["FIDC", "FIP", "F.I.I."])
    ]
    
    print(f"🔍 Total de arquivos PDF encontrados: {len(lista_pdfs)}")

    # lista_pdfs =#lista_pdfs[:50] #processa x pdfs para teste
    #print(f"🔧 Modo teste: processando apenas {len(lista_pdfs)} arquivos...")

    # Lista para armazenar todos os dados extraídos
    todos_dados = []
    
    # Processar cada PDF
    for i, pdf in enumerate(lista_pdfs, 1):
        print(f"\n📄 ({i}/{len(lista_pdfs)}) Processando: {os.path.basename(pdf)}")
        
        # Extrair CNPJ e data do nome do arquivo
        tipo, cnpj_arquivo, data_arquivo = extrair_info_arquivo(pdf)
        

        # Extrair texto do PDF
        texto_completo = extrair_texto_pdf(pdf)
        dados = extrai_infos_prospectos(texto_completo)
            
        # Adicionar metadados (CNPJ e data)
        dados["tipo"]= tipo
        dados["cnpj"] = cnpj_arquivo
        dados["data_emissao"] = data_arquivo
        todos_dados.append(dados)
    
    # Persistência dos dados no Databricks
    if todos_dados:
        inserir_dados_databricks(todos_dados)
        print(f"✅ Dados inseridos com sucesso! Total: {len(todos_dados)} registros.")
    else:
        print("⚠️ Nenhum dado foi extraído com sucesso.")       

def combinar_dados(dados_parte1, dados_parte2):
    """
    Combina os dados de duas extrações, priorizando a primeira parte
    e usando a segunda parte apenas para preencher campos vazios.
    """
    dados_finais = dados_parte1.copy()
    
    campos_preenchidos = 0
    
    for chave, valor_parte1 in dados_parte1.items():
        # Se o campo está vazio na primeira parte, tentar preencher com a segunda
        if not valor_parte1.strip():  # Campo vazio ou só espaços
            if chave in dados_parte2 and dados_parte2[chave].strip():
                dados_finais[chave] = dados_parte2[chave]
                campos_preenchidos += 1
                print(f"  → Campo '{chave}' preenchido com dados da segunda parte")
    
    print(f"Total de campos preenchidos pela segunda parte: {campos_preenchidos}")
    
    return dados_finais
def inserir_dados_databricks(lista_dados):
    """
    Insere os dados extraídos na tabela do Databricks
    """
    print(f"\nInserindo {len(lista_dados)} registros na tabela...")
    
    # Definir o schema da tabela
    schema = StructType([
         StructField("tipo", StringType(), True),
        StructField("cnpj", StringType(), True),
        StructField("data_emissao", StringType(), True),
        StructField("qt_emissoes", StringType(), True),
        StructField("nome_fundo", StringType(), True),
        StructField("valor_cota_emissao", StringType(), True),
        StructField("direito_preferencia_sobras_montante_adicional", StringType(), True),
        StructField("taxa_distribuicao_emissao", StringType(), True),
        StructField("tabela_ativos_fundo", StringType(), True),
        StructField("sumario_experiencia_socios", StringType(), True),
        StructField("quantidade_cotas_emissao", StringType(), True),
        StructField("quantidade_cotas_adicionais_emissao", StringType(), True),
        StructField("publico_alvo", StringType(), True),
        StructField("obs_publico_alvo", StringType(), True),
        StructField("procuracao_AGE", StringType(), True),
        StructField("planilha_custos", StringType(), True),
        StructField("ordenar_fatores_risco", StringType(), True),
        StructField("montante_minimo_emissao", StringType(), True),
        StructField("investimento_minimo_cpf_cnpj", StringType(), True),
        StructField("investimento_minimo_inst", StringType(), True),
        StructField("investimento_maximo_cpf_cnpj", StringType(), True),
        StructField("investimento_maximo_inst", StringType(), True),
        StructField("historico_cotacao_bolsa", StringType(), True),
        StructField("fator_proporcao_dp", StringType(), True),
        StructField("diluicao_economica_novas_emissoes", StringType(), True),
        StructField("criterio_rateio", StringType(), True),
        StructField("carteira_fundos_kinea_intrag", StringType(), True),
        StructField("breve_historico_gestor", StringType(), True),
        StructField("percentual_oferta_institucional", StringType(), True),
        StructField("volume_base_emissao", StringType(), True),
        StructField("chamada_capital_ipca", StringType(), True)
    ])
    
    # Garantir que todos os campos da tabela estão presentes
    dados_para_inserir = []
    for dados in lista_dados:
        dados_completos = {}
        for field in schema.fields:
            dados_completos[field.name] = dados.get(field.name, "")
        dados_para_inserir.append(dados_completos)
    
    # Criar DataFrame e inserir
    df = spark.createDataFrame(dados_para_inserir, schema)
    df.write.mode("append").saveAsTable("desafio_kinea.prospecto_fundos.extracao_prospectos_v3")
    
    print(f"✅ {len(lista_dados)} registros inseridos com sucesso!")
        
           
if __name__ == "__main__":
    main()




In [0]:
%sql
SELECT * FROM desafio_kinea.prospecto_fundos.extracao_prospectos_v3
WHERE valor_cota_emissao = ""

LLM para ler colunas

In [0]:
import pdfplumber
import openai
import json
import os
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

def extrair_texto_colunas_pdf(caminho_pdf):
    texto_total = ""
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            largura = pagina.width
            altura = pagina.height

            # Divide em colunas
            col_esq = pagina.crop((0, 0, largura / 2, altura)).extract_text() or ""
            col_dir = pagina.crop((largura / 2, 0, largura, altura)).extract_text() or ""

            texto_total += col_esq + "\n" + col_dir + "\n\n"
    
    return texto_total.strip()


def extrai_infos_prospectos(texto):
    prompt = f"""Role: Act as a highly specialized AI document analyst trained in processing and extracting structured data from official financial documents, specifically related to Investment Funds in Brazil.

    Context:
    You will receive the full text of a regulatory or offering document related to a quota issuance (e.g., prospecto). These documents are usually published in PDF format and contain financial, legal, and operational details about a fund offering.

    Your Objective:
    Extract ** this specific fields** from the document content. Each field corresponds to a relevant piece of information necessary for evaluating the offering. If multiple mentions of the same data point exist, prioritize:
    1. The version labeled as definitive or part of the final offer terms (e.g., \"condições da oferta\").
    2. The most recent or prominent value.
    3. If you are not sure of the information, it's better to return an empty string.
    

    Output Instructions:
    – **Do not infer or fabricate information**. If a field is not explicitly mentioned or lacks clarity, return an **empty string** \"\".
    – All numerical values must use the **international format**: use . (dot) for decimals and **no thousand separators**.
    – Return the result as a **strictly valid JSON object**, conforming exactly to the schema below.
    – **Do not include any explanatory text**, headers, comments, or notes outside the JSON object.
    – The order of keys in the output must match the list below.
    – If a field is a list or summary, return a string.

    
    Constraints:
    – Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string.
    – Use Brazilian Portuguese language context for comprehension but ensure numeric formatting adheres to international standards.
    – Be precise and concise. Avoid speculative language.
    

    You are expected to handle varied writing styles and formatting typically found in official financial documents and maintain high accuracy under ambiguity.
    Informações a extrair (nessa ordem):
        1. CNPJ do fundo
        2. Data de emissão do fundos
        3. Quantidade de emissões que aquele fundo já passou
        4. Nome do Fundo
        5. Valor da Cota da Emissão
        6. Se existe direito de preferência, sobras ou montante adicional
        7. Taxa de distribuição da Emissão
        8. Tabela de ativos do fundo
        9. Sumário de experiência dos sócios
        10. Quantidade de cotas da Emissão
        11. Quantidade de cotas adicionais na Emissão
        12. Público-alvo
        13. Procuração para AGE (se necessária)
        14. Planilha de custos
        15. Ordenação dos fatores de risco
        16. Montante mínimo da Emissão
        17. Investimento mínimo por CPF/CNPJ
        18. Investimento mínimo para investidores institucionais
        19. Investimento máximo por CPF/CNPJ
        20. Investimento máximo para investidores institucionais
        21. Histórico de cotação em bolsa (mínima, média e máxima)
        22. Fator de proporção para o Direito de Preferência (DP)
        23. Diluição econômica em novas emissões
        24. Critério de rateio
        25. Breve histórico do gestor
        26. % da oferta destinada ao público institucional
        27. Volume base da Emissão
    """
    message_text = [
        {'role': 'system', 'content': prompt},
        {"role": "user", "content": texto}
    ]

    func = {
        'name': 'extrair_informacao_pdf',
        'description': 'Função para extrair os valores do PDF.',
        'parameters': {
            "type": "object",
            "properties": {
                "cnpj": {
                "type": "string",
                "description": "CNPJ do fundo. Extrair informação no formato padrão brasileiro (ex: \"60.431.592/0001-28\")."
                },
                "data_emissao": {
                "type": "string",
                "description": "Data de emissão do fundo. A informação pode estar no texto com formato ex: 'A data deste Prospecto Definitivo é 23 de junho de 2025'. Extraia a a data no formato YYYY-MM-DD (ex: \"2025-06-23\"), caso não tenha a data exata pode apresentar apenas no formato YYYY-MM."
                },
                "qt_emissoes": {
                "type": "string",
                "description": "Quantidade de emissões que aquele fundo já passou. Informe como número (ex.: \"1\", \"2\", etc.). A informação pode estar no formato 'PROSPECTO DEFINITIVO DE DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 1ª (PRIMEIRA) EMISSÃO...' ou então 'DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 4ª (QUARTA) EMISSÃO...'." 
                },
                "nome_fundo": {
                "type": "string",
                "description": "Extrair o nome do fundo"
                },
                "valor_cota_emissao": {
                "type": "string",
                "description": """Preço unitário de subscrição por cota na nova emissão. Informe como número (ex.: \"98.45\") ou texto vazio se não disponível.
                Consultar texto introdutório do prospecto. Informação pode estar no formato: "...Serão emitidas inicialmente cotas no valor de R$10,00 (dez reais) cada, independentemente da classe..." Caso não exista, retornar string vazia."""
                },
                "direito_preferencia_sobras_montante_adicional": {
                "type": "string",
                "description": """Indique se há ou não direito de preferência, possibilidade de sobras ou de montante adicional. Ex.: \"Sim\" ou \"Não\".
                A informação pode estar na seção com nome "Informações sobre a existência de direito de preferência na subscrição de novas cotas" ou similar. Caso não exista, retornar string vazia."""
                },
                "taxa_distribuicao_emissao": {
                "type": "string",
                "description": "Percentual ou valor fixo da taxa de distribuição primária (remuneração do coordenador líder). Ex.: \"3.0%\". A informação pode estar numa tabela seguida do número por extenso. Extraia o valor do percentual em relação ao valor da cota ou então o percentual em relação ao montante total. Ex: 2,00% (dois por cento) . Caso não exista, retornar string vazia."
                },
                "tabela_ativos_fundo": {
                "type": "string",
                "description": """Cópia ou referência resumida da tabela/lista dos principais ativos que compõem a carteira-alvo ou carteira atual do fundo. Liste os ativos somente com os nomes deles (seguidos da sigla se tiver ex: CRI) separados por vírgula. Não é necessário numerar. A informação pode estar na seção 'Destinação dos Recursos', 'Política de Investimentos', 'Composição da Carteira', 'Objetivo do Fundo' ou similar. A informação pode estar no formato '...este sentido, os Ativos nos quais o Fundo poderá investir são: (i) Certificados de Recebíveis Imobiliários ("CRI"); (ii) cotas de fundos de investimento imobiliário ("FII"); (iii) debêntures ("Debêntures") emitidas por emissores devidamente autorizados nos termos da regulamentação aplicável, e cujas atividades preponderantes sejam permitidas aos FII... Caso não exista, retornar string vazia."""
                },
                "sumario_experiencia_socios": {
                "type": "string",
                "description": "Síntese da experiência profissional dos sócios/gestores apresentada no documento. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_emissao": {
                "type": "string",
                "description": "Quantidade total de cotas ofertadas na emissão base (sem considerar lotes adicionais). Ex.: \"5.000.000\". Consultar texto inicial do prospecto. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_adicionais_emissao": {
                "type": "string",
                "description": "Quantidade de cotas do lote adicional ou suplementar, se previsto. Consultar texto inicial do prospecto. A informação pode estar no formato '...a quantidade de Cotas inicialmente ofertada poderá ser acrescida em até 25% (vinte e cinco por cento), ou seja, em até 7.500.000 (sete milhões e quinhentas mil) Cotas...' . Caso não exista, retornar string vazia."
                },
                "publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta (ex.: \"Investidores em geral\", \"Investidores profissionais\", etc.). A informação pode estar na seção 'Identificação do público alvo' ou similar. Caso não exista, retornar string vazia."
                },
                "procuracao_AGE": {
                "type": "string",
                "description": "Indicação se o documento contém modelo de procuração para AGE ou se não é necessária. A informação pode estar geralmente no final do prospecto ou em seção chamada 'Procuração', 'Assembleia Geral', 'Anexos1' ou similar. Ex.: \"Modelo de procuração incluso\" ou \"Não aplicável\".Caso não exista, retornar string vazia."
                },
                "planilha_custos": {
                "type": "string",
                "description": "Referência ou resumo da planilha de custos da oferta (incluindo taxas, despesas, previsão de custos). A informação pode estar na seção  'Custos da Distribuição', 'Demonstrativo de Custos', 'Contrato de Distribuição'. discriminando: a) a porcentagem em relação ao preço unitário de subscrição; b) a comissão de coordenação; c) a comissão de distribuição; d) a comissão de garantia de subscrição, se houver; e) outras comissões (especificar); f) os tributos incidentes sobre as comissões, caso estes sejam arcados pela classe de cotas; g) o custo unitário de distribuição; h) as despesas decorrentes do registro de distribuição; e i) outros custos relacionados ou similar. Ex: \"a) comissão de distribuição: 2.0%... b) comissão de coordenação: 0.5%...\". Caso não exista, retornar string vazia. "
                },
                "ordenar_fatores_risco": {
                "type": "string",
                "description": "Fatores de risco já em ordem apresentada ou indicação de onde ela se encontra. Indicar o nome do risco seguido da sua intensidade ex: Risco de Crédito (Maior). Não use listas, somente strings simples.A informação pode estar na seção 'Fatores de Risco' ou similar.Caso não exista, retornar string vazia."
                },
                "montante_minimo_emissao": {
                "type": "string",
                "description": "Valor mínimo da oferta para que a emissão seja considerada válida (montante mínimo de liquidação). A informação pode estar na seção 'Distribuição Parcial', 'Montante Mínimo', 'Características da Oferta' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_cpf_cnpj": {
                "type": "string",
                "description": "Valor mínimo de subscrição por investidor pessoa física ou jurídica (CPF/CNPJ). A informação pode estar na seção 'Aplicação Mínima', 'Investimento Mínimo', 'Público Alvo' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_inst": {
                "type": "string",
                "description": "Valor mínimo de subscrição exigido para investidores institucionais, se diferente. A informação pode estar na seção  'Público-alvo', 'Distribuição para Institucionais' ou similar. Consulte para saber se investidores institucionais são contemplados. Se sim, extrair o valor mínimo de investimento para esse caso. Caso não exista, retornar string vazia."
                },
                "investimento_maximo_cpf_cnpj": {
                "type": "string",
                "description": "Valor máximo permitidos por investidor pessoa física ou jurídica (CPF/CNPJ), se houver restrição. A informação pode estar na seções 'Público-Alvo' ou cláusulas com restrições de alocação ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_maximo_inst": {
                "type": "string",
                "description": "Valor máximo permitido para investidores institucionais, se houver. A informação pode estar na seção  'Público-alvo', 'Distribuição para Institucionais' ou similar.Caso não exista, retornar string vazia."
                },
                "historico_cotacao_bolsa": {
                "type": "string",
                "description": "Valores mínima, média e máxima da cota em bolsa conforme apresentado (ex.: \"Mín 90.10 / Méd 94.50 / Máx 98.00\").A informação pode estar na seção 'Cotação em Bolsa', 'Informações sobres as cotas' ou similar.Caso não exista, retornar string vazia."
                },
                "fator_proporcao_dp": {
                "type": "string",
                "description": "Fator de proporção do Direito de Preferência (DP). A informação pode estar na seção 'Distribuição Parcial', 'Lote Adicional', 'Rateio de Cotas', ou na parte que trata do 'Plano de Distribuição'. Extraia a proporção, se existir, no mesmo formato descrito no documento.Caso não exista, retornar string vazia."
                },
                "diluicao_economica_novas_emissoes": {
                "type": "string",
                "description": "Percentual estimado de diluição econômica para cotistas atuais após a nova emissão, ou texto explicativo.Caso não exista, retornar string vazia."
                },
                "criterio_rateio": {
                "type": "string",
                "description": "Critério de rateio em caso de excesso de demanda. A informação pode estar na seção 'Critério de Rateio', 'Outras Características da Oferta' ou similar. Ex.: \"Pro rata\" ou \"Prioridade investidores profissionais\".Caso não exista, retornar string vazia."
                },
                "percentual_oferta_institucional": {
                "type": "string",
                "description": "Percentual da oferta reservado ao público institucional. Ex.: \"30%\".Caso não exista, retornar string vazia."
                },
                "volume_base_emissao": {
                "type": "string",
                "description": "Montante financeiro (R$) correspondente ao volume base da emissão sem considerar lote adicional.Caso não exista, retornar string vazia."
                },
                "chamada_capital_ipca": {
                "type": "string",
                "description": "Descrição de cláusula de chamada de capital com atualização pelo IPCA, se existente. Caso não exista, retornar string vazia."
                }
            },
            "required": [
                "cnpj",
                "data_emissao",
                "qt_emissoes",
                "nome_fundo",
                "valor_cota_emissao",
                "direito_preferencia_sobras_montante_adicional",
                "taxa_distribuicao_emissao",
                "tabela_ativos_fundo",
                "sumario_experiencia_socios",
                "quantidade_cotas_emissao",
                "quantidade_cotas_adicionais_emissao",
                "publico_alvo",
                "obs_publico_alvo",
                "procuracao_AGE",
                "planilha_custos",
                "ordenar_fatores_risco",
                "montante_minimo_emissao",
                "investimento_minimo_cpf_cnpj",
                "investimento_minimo_inst",
                "investimento_maximo_cpf_cnpj",
                "investimento_maximo_inst",
                "historico_cotacao_bolsa",
                "fator_proporcao_dp",
                "diluicao_economica_novas_emissoes",
                "criterio_rateio",
                "breve_historico_gestor",
                "percentual_oferta_institucional",
                "volume_base_emissao",
                "chamada_capital_ipca"
            ]
        }
    }

    try:
        completion = openai.ChatCompletion.create(
            engine="gpt-4.1-mini",  
            messages=message_text,
            temperature=0.1,
            top_p=0.95,
            tools=[{"type": "function", "function": func}],
            tool_choice={"type": "function", "function": {"name": func["name"]}}
        )

        responseText = completion.to_dict()['choices'][0]['message']
        arguments_str = responseText['tool_calls'][0]["function"]["arguments"]
        
        # Parse do JSON retornado pela função
        dados_extraidos = json.loads(arguments_str)
        
        return dados_extraidos
        
    except json.JSONDecodeError as e:
        print(f"Erro ao fazer parse do JSON: {e}")
        return {"erro": "Falha ao processar resposta da IA - JSON inválido"}
    
    except KeyError as e:
        print(f"Erro de estrutura na resposta: {e}")
        return {"erro": "Falha na estrutura da resposta da IA"}
        
    except Exception as e:
        print(f"Erro geral: {e}")
        return {"erro": f"Erro inesperado: {str(e)}"}
                


def inserir_dados_databricks(lista_dados):
    """
    Insere os dados extraídos na tabela do Databricks
    """
    print(f"\nInserindo {len(lista_dados)} registros na tabela...")
    
    # Definir o schema da tabela
    schema = StructType([
        StructField("cnpj", StringType(), True),
        StructField("data_emissao", StringType(), True),
        StructField("qt_emissoes", StringType(), True),
        StructField("nome_fundo", StringType(), True),
        StructField("valor_cota_emissao", StringType(), True),
        StructField("direito_preferencia_sobras_montante_adicional", StringType(), True),
        StructField("taxa_distribuicao_emissao", StringType(), True),
        StructField("tabela_ativos_fundo", StringType(), True),
        StructField("sumario_experiencia_socios", StringType(), True),
        StructField("quantidade_cotas_emissao", StringType(), True),
        StructField("quantidade_cotas_adicionais_emissao", StringType(), True),
        StructField("publico_alvo", StringType(), True),
        StructField("obs_publico_alvo", StringType(), True),
        StructField("procuracao_AGE", StringType(), True),
        StructField("planilha_custos", StringType(), True),
        StructField("ordenar_fatores_risco", StringType(), True),
        StructField("montante_minimo_emissao", StringType(), True),
        StructField("investimento_minimo_cpf_cnpj", StringType(), True),
        StructField("investimento_minimo_inst", StringType(), True),
        StructField("investimento_maximo_cpf_cnpj", StringType(), True),
        StructField("investimento_maximo_inst", StringType(), True),
        StructField("historico_cotacao_bolsa", StringType(), True),
        StructField("fator_proporcao_dp", StringType(), True),
        StructField("diluicao_economica_novas_emissoes", StringType(), True),
        StructField("criterio_rateio", StringType(), True),
        StructField("carteira_fundos_kinea_intrag", StringType(), True),
        StructField("breve_historico_gestor", StringType(), True),
        StructField("percentual_oferta_institucional", StringType(), True),
        StructField("volume_base_emissao", StringType(), True),
        StructField("chamada_capital_ipca", StringType(), True)
    ])
    
    # Garantir que todos os campos da tabela estão presentes
    dados_para_inserir = []
    for dados in lista_dados:
        dados_completos = {}
        for field in schema.fields:
            # Preenche com string vazia se o campo não existir nos dados extraídos
            dados_completos[field.name] = dados.get(field.name, "")
        dados_para_inserir.append(dados_completos)
    
    # Criar DataFrame e inserir
    df = spark.createDataFrame(dados_para_inserir, schema)
    df.write.format("delta").mode("append").saveAsTable("desafio_kinea.prospecto_fundos.extracao_prospectos_v2")
    
    print(f"✅ {len(lista_dados)} registros inseridos com sucesso!")

def main():
    # Lista de PDFs a serem processados
    lista_pdfs = [
        "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/FIDC_06018364000185_20060630.pdf",
        # Adicione outros PDFs aqui...
    ]

    todos_dados = []

    for caminho_pdf in lista_pdfs:
        print(f"\n📄 Processando: {os.path.basename(caminho_pdf)}")
        
        # Extrai texto (com tratamento de colunas)
        texto = extrair_texto_colunas_pdf(caminho_pdf)
        
        # Envia para a OpenAI
        dados = extrai_infos_prospectos(texto)
        
        if dados and not dados.get("erro"):
            todos_dados.append(dados)
            print("✅ Dados extraídos com sucesso!")
            # Mostrar preview dos dados
            for chave, valor in list(dados.items())[:5]:  # Mostra apenas os 5 primeiros campos
                print(f"  {chave}: {valor[:100]}{'...' if len(str(valor)) > 100 else ''}")
        else:
            print(f"❌ Falha na extração: {dados.get('erro', 'Erro desconhecido')}")

    # Persiste no Databricks
    if todos_dados:
        inserir_dados_databricks(todos_dados)
    else:
        print("Nenhum dado válido para inserir.")

if __name__ == "__main__":
    main()

6: Adotando a estratégia de dividir os pdfs em duas partes para melhor processamento da LLM.
- código para a criação da tabela
- código para a extração das informações



In [0]:
%sql
DROP TABLE desafio_kinea.prospecto_fundos.extracao_prospectos_kinea_v2

In [0]:
%sql
CREATE OR REPLACE TABLE desafio_kinea.prospecto_fundos.extracao_prospectos_kinea_v2 (
  cnpj STRING,
  data_emissao STRING,
  qt_emissoes STRING,
  nome_fundo STRING,
  valor_cota_emissao STRING,
  direito_preferencia_sobras_montante_adicional STRING,
  taxa_distribuicao_emissao STRING,
  tabela_ativos_fundo STRING,
  sumario_experiencia_socios STRING,
  quantidade_cotas_emissao STRING,
  quantidade_cotas_adicionais_emissao STRING,
  publico_alvo STRING,
  obs_publico_alvo STRING,
  procuracao_AGE STRING,
  planilha_custos STRING,
  ordenar_fatores_risco STRING,  -- Campo ajustado para STRING
  montante_minimo_emissao STRING,
  investimento_minimo_cpf_cnpj STRING,
  investimento_minimo_inst STRING,
  investimento_maximo_cpf_cnpj STRING,
  investimento_maximo_inst STRING,
  historico_cotacao_bolsa STRING,
  fator_proporcao_dp STRING,
  diluicao_economica_novas_emissoes STRING,
  criterio_rateio STRING,
  carteira_fundos_kinea_intrag STRING,
  breve_historico_gestor STRING,
  percentual_oferta_institucional STRING,
  volume_base_emissao STRING,
  chamada_capital_ipca STRING
)
USING DELTA;



CÓDIGO PARA EXTRAIR AS INFORMAÇÕES ATUALIZADAS. MÉTODO DE DIVIDIR O TEXTO DO PDF EM 2.

In [0]:
import fitz
import base64
import re
import os
import openai
import tempfile
import json
from datetime import datetime
from functools import lru_cache
from json import dumps
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

def dividir_pdf_em_duas_partes(caminho_pdf):
    doc = fitz.open(caminho_pdf)
    total_paginas = len(doc)
    metade = total_paginas // 2

    texto_parte1 = ""
    texto_parte2 = ""

    for i in range(total_paginas):
        texto = doc[i].get_text()
        if i < metade:
            texto_parte1 += texto
        else:
            texto_parte2 += texto

    doc.close()
    return texto_parte1.strip(), texto_parte2.strip()

# A função a seguir:
# - recebe um texto (no caso o PDF convertido)
# - retorna um dicionário com as informações extraídas.

def extrai_infos(texto):
    prompt = f"""Role: Act as a highly specialized AI document analyst trained in processing and extracting structured data from official financial documents, specifically related to Investment Funds in Brazil.

    Context:
    You will receive the full text of a regulatory or offering document related to a quota issuance (e.g., prospecto). These documents are usually published in PDF format and contain financial, legal, and operational details about a fund offering.

    Your Objective:
    Extract ** this specific fields** from the document content. Each field corresponds to a relevant piece of information necessary for evaluating the offering. If multiple mentions of the same data point exist, prioritize:
    1. The version labeled as definitive or part of the final offer terms (e.g., \"condições da oferta\").
    2. The most recent or prominent value.
    3. If you are not sure of the information, it's better to return an empty string.
    

    Output Instructions:
    – **Do not infer or fabricate information**. If a field is not explicitly mentioned or lacks clarity, return an **empty string** `\"\"`.
    – All numerical values must use the **international format**: use `.` (dot) for decimals and **no thousand separators**.
    – Return the result as a **strictly valid JSON object**, conforming exactly to the schema below.
    – **Do not include any explanatory text**, headers, comments, or notes outside the JSON object.
    – The order of keys in the output must match the list below.
    – If a field is a list or summary, return a string.

    
    Constraints:
    – Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string.
    – Use Brazilian Portuguese language context for comprehension but ensure numeric formatting adheres to international standards.
    – Be precise and concise. Avoid speculative language.
    

    You are expected to handle varied writing styles and formatting typically found in official financial documents and maintain high accuracy under ambiguity.
    Informações a extrair (nessa ordem):
        1. CNPJ do fundo
        2. Data de emissão do fundos
        3. Quantidade de emissões que aquele fundo já passou
        4. Nome do Fundo
        5. Valor da Cota da Emissão
        6. Se existe direito de preferência, sobras ou montante adicional
        7. Taxa de distribuição da Emissão
        8. Tabela de ativos do fundo
        9. Sumário de experiência dos sócios
        10. Quantidade de cotas da Emissão
        11. Quantidade de cotas adicionais na Emissão
        12. Público-alvo
        13. Procuração para AGE (se necessária)
        14. Planilha de custos
        15. Ordenação dos fatores de risco
        16. Montante mínimo da Emissão
        17. Investimento mínimo por CPF/CNPJ
        18. Investimento mínimo para investidores institucionais
        19. Investimento máximo por CPF/CNPJ
        20. Investimento máximo para investidores institucionais
        21. Histórico de cotação em bolsa (mínima, média e máxima)
        22. Fator de proporção para o Direito de Preferência (DP)
        23. Diluição econômica em novas emissões
        24. Critério de rateio
        25. Carteira de fundos Kinea com Intrag
        26. Breve histórico do gestor
        27. % da oferta destinada ao público institucional
        28. Volume base da Emissão
    """
    message_text = [
        {'role': 'system', 'content': prompt},
        {"role": "user", "content": texto}
    ]

    func = {
        'name': 'extrair_informacao_pdf',
        'description': 'Função para extrair os valores do PDF.',
        'parameters': {
            "type": "object",
            "properties": {
                "cnpj": {
                "type": "string",
                "description": "CNPJ do fundo. Extrair informação no formato padrão brasileiro (ex: \"60.431.592/0001-28\")."
                },
                "data_emissao": {
                "type": "string",
                "description": "Data de emissão do fundo. A informação pode estar no texto com formato ex: 'A data deste Prospecto Definitivo é 23 de junho de 2025'. Extraia a a data no formato YYYY-MM-DD (ex: \"2025-06-23\")."
                },
                "qt_emissoes": {
                "type": "string",
                "description": "Quantidade de emissões que aquele fundo já passou. Informe como número (ex.: \"1\", \"2\", etc.). A informação pode estar no formato 'PROSPECTO DEFINITIVO DE DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 1ª (PRIMEIRA) EMISSÃO...' ou então 'DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 4ª (QUARTA) EMISSÃO...'." 
                },
                "nome_fundo": {
                "type": "string",
                "description": "Extrair o nome do fundo"
                },
                "valor_cota_emissao": {
                "type": "string",
                "description": """Preço unitário de subscrição por cota na nova emissão. Informe como número (ex.: \"98.45\") ou texto vazio se não disponível.
                Consultar texto introdutório do prospecto. Informação pode estar no formato: "...nominativas e escriturais, da 4ª Emissão ("4ª Emissão" e "Novas Cotas"), pelo valor unitário de R$ 101,26 (cento e um reais e vinte e seis centavos), correspondente ao valor patrimonial das cotas da Classe... Caso não exista, retornar string vazia."""
                },
                "direito_preferencia_sobras_montante_adicional": {
                "type": "string",
                "description": """Indique se há ou não direito de preferência, possibilidade de sobras ou de montante adicional. Ex.: \"Sim\" ou \"Não\".
                A informação pode estar na seção com nome "Informações sobre a existência de direito de preferência na subscrição de novas cotas" ou similar. Caso não exista, retornar string vazia."""
                },
                "taxa_distribuicao_emissao": {
                "type": "string",
                "description": "Percentual ou valor fixo da taxa de distribuição primária (remuneração do coordenador líder). Ex.: \"3.0%\". A informação pode estar numa tabela seguida do número por extenso. Extraia o valor do percentual em relação ao valor da cota ou então o percentual em relação ao montante total. Ex: 2,00% (dois por cento) . Caso não exista, retornar string vazia."
                },
                "tabela_ativos_fundo": {
                "type": "string",
                "description": """Cópia ou referência resumida da tabela/lista dos principais ativos que compõem a carteira-alvo ou carteira atual do fundo.Liste os ativos somente com os nomes deles (seguidos da sigla se tiver ex: CRI) separados por vírgula. Não é necessário numerar. A informação pode estar na seção 'Destinação dos Recursos' ou similar. A informação pode estar no formato '...este sentido, os Ativos nos quais o Fundo poderá investir são: (i) Certificados de Recebíveis Imobiliários ("CRI"); (ii) cotas de fundos de investimento imobiliário ("FII"); (iii) debêntures ("Debêntures") emitidas por emissores devidamente autorizados nos termos da regulamentação aplicável, e cujas atividades preponderantes sejam permitidas aos FII... Caso não exista, retornar string vazia."""
                },
                "sumario_experiencia_socios": {
                "type": "string",
                "description": "Síntese da experiência profissional dos sócios/gestores apresentada no documento. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_emissao": {
                "type": "string",
                "description": "Quantidade total de cotas ofertadas na emissão base (sem considerar lotes adicionais). Ex.: \"5.000.000\". Consultar texto inicial do prospecto. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_adicionais_emissao": {
                "type": "string",
                "description": "Quantidade de cotas do lote adicional ou suplementar, se previsto. Consultar texto inicial do prospecto. A informação pode estar no formato '... Os Ofertantes, nos termos e conforme os limites estabelecidos no artigo 50 da Resolução CVM nº 160, com a prévia concordância do Coordenador Líder (conforme abaixo definido), poderão optar por distribuir um volume adicional de até 25% (vinte e cinco por cento) da quantidade máxima de Novas Cotas inicialmente ofertadas, ou seja, até 1.234.445 (um milhão, duzentas e trinta e quatro mil, quatrocentas e quarenta e cinco) Novas Cotas...' . Caso não exista, retornar string vazia."
                },
                "publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta. A informação pode estar na seção 'Identificação do público alvo' ou similar. Retorne entre as opções possíveis: \"Investidores em geral\" , \"Investidores qualificados\", \"Investidores Institucionais\". Se a informação não está presente no texto, retorne \"Investidores em geral\" como padrão. "
                },
                "obs_publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta. A informação pode estar na seção 'Identificação do público alvo' ou similar. Consulte esse fragmento de texto para extrair se, além de Investidores em geral/qualificados/institucionais, existem outras restrições. Retorne somente os casos especiais (ex: \"Clientes Itaú Unibanco\").Caso não exista, retornar string vazia."
                },
                "procuracao_AGE": {
                "type": "string",
                "description": "Indicação se o documento contém modelo de procuração para AGE ou se não é necessária. A informação pode estar na seção 'Indicar a eventual possibilidade de destinação dos recursos a quaisquer ativos em relação às quais possa haver conflito de interesse, informando as aprovações necessárias existentes e/ou a serem obtidas, incluindo nesse caso nos fatores de risco, explicação objetiva sobre a falta de transparência na formação dos preços destas operações' ou similar. Ex.: \"Modelo de procuração incluso\" ou \"Não aplicável\".Caso não exista, retornar string vazia."
                },
                "planilha_custos": {
                "type": "string",
                "description": "Referência ou resumo da planilha de custos da oferta (taxas, despesas, previsão de custos). A informação pode estar na seção 'Demonstrativo do custo da distribuição, discriminando: a) a porcentagem em relação ao preço unitário de subscrição; b) a comissão de coordenação; c) a comissão de distribuição; d) a comissão de garantia de subscrição, se houver; e) outras comissões (especificar); f) os tributos incidentes sobre as comissões, caso estes sejam arcados pela classe de cotas; g) o custo unitário de distribuição; h) as despesas decorrentes do registro de distribuição; e i) outros custos relacionados' ou similar. Ex: \"Taxa de distribuição total 2.00%, comissão de coordenação 0.10%, comissão de distribuição 1.70%, advogados 0.04%, tributos advogados 0.00%, taxa registro CVM 0.04%, taxa registro Anbima 0.00%, taxa B3 distribuição padrão fixa 0.03%, taxa B3 distribuição padrão variável 0.04%, taxa B3 análise oferta 0.00%, taxa B3 listagem 0.00%, anúncio 0.00%, cartório 0.00%, outras despesas 0.02%.\"Caso não exista, retornar string vazia. "
                },
                "ordenar_fatores_risco": {
                "type": "string",
                "description": "Fatores de risco já em ordem apresentada ou indicação de onde ela se encontra. Indicar o nome do risco seguido da sua intensidade ex: Risco de Crédito (Maior). Não use listas, somente strings simples.A informação pode estar na seção 'Fatores de Risco' ou similar.Caso não exista, retornar string vazia."
                },
                "montante_minimo_emissao": {
                "type": "string",
                "description": "Valor mínimo da oferta para que a emissão seja considerada válida (montante mínimo de liquidação). A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_cpf_cnpj": {
                "type": "string",
                "description": "Valor mínimo de subscrição por investidor pessoa física ou jurídica (CPF/CNPJ).A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_inst": {
                "type": "string",
                "description": "Valor mínimo de subscrição exigido para investidores institucionais, se diferente. A informação pode estar na seção  'Identificação do público-alvo' ou similar. Consulte para saber se investidores institucionais são contemplados. Se sim, extrair o valor mínimo de investimento para esse caso. Caso não exista, retornar string vazia."
                },
                "investimento_maximo_cpf_cnpj": {
                "type": "string",
                "description": "Valor máximo permitidos por investidor pessoa física ou jurídica (CPF/CNPJ), se houver restrição. A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_maximo_inst": {
                "type": "string",
                "description": "Valor máximo permitido para investidores institucionais, se houver. A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "historico_cotacao_bolsa": {
                "type": "string",
                "description": "Valores mínima, média e máxima da cota em bolsa conforme apresentado (ex.: \"Mín 90.10 / Méd 94.50 / Máx 98.00\").A informação pode estar na seção 'Cotação em bolsa de valores ou mercado de balcão dos valore mobiliários a serem distribuídos, inclusive no exterior' ou similar.Caso não exista, retornar string vazia."
                },
                "fator_proporcao_dp": {
                "type": "string",
                "description": "Fator de proporção do Direito de Preferência (DP). A informação pode estar na seção 'Informações sobre a existência de direito de preferência na subscrição de novas cotas'. Extraia a proporção, se existir, no mesmo formato descrito no documento.Caso não exista, retornar string vazia."
                },
                "diluicao_economica_novas_emissoes": {
                "type": "string",
                "description": "Percentual estimado de diluição econômica para cotistas atuais após a nova emissão, ou texto explicativo.Caso não exista, retornar string vazia."
                },
                "criterio_rateio": {
                "type": "string",
                "description": "Critério de rateio em caso de excesso de demanda. A informação pode estar na seção 'Critério de Rateio' ou similar. Ex.: \"Pro rata\" ou \"Prioridade investidores profissionais\".Caso não exista, retornar string vazia."
                },
                "carteira_fundos_kinea_intrag": {
                "type": "string",
                "description": "Informação específica sobre a carteira de fundos Kinea com Intrag, se mencionada.Caso não exista, retornar string vazia."
                },
                "breve_historico_gestor": {
                "type": "string",
                "description": "Resumo do histórico da gestora ou do gestor responsável conforme descrito no documento.Caso não exista, retornar string vazia."
                },
                "percentual_oferta_institucional": {
                "type": "string",
                "description": "Percentual da oferta reservado ao público institucional. Ex.: \"30%\".Caso não exista, retornar string vazia."
                },
                "volume_base_emissao": {
                "type": "string",
                "description": "Montante financeiro (R$) correspondente ao volume base da emissão sem considerar lote adicional.Caso não exista, retornar string vazia."
                },
                "chamada_capital_ipca": {
                "type": "string",
                "description": "Descrição de cláusula de chamada de capital com atualização pelo IPCA, se existente. Caso não exista, retornar string vazia."
                }
            },
            "required": [
                "cnpj",
                "data_emissao",
                "qt_emissoes",
                "nome_fundo"
                "valor_cota_emissao",
                "direito_preferencia_sobras_montante_adicional",
                "taxa_distribuicao_emissao",
                "tabela_ativos_fundo",
                "sumario_experiencia_socios",
                "quantidade_cotas_emissao",
                "quantidade_cotas_adicionais_emissao",
                "publico_alvo",
                "obs_publico_alvo",
                "procuracao_AGE",
                "planilha_custos",
                "ordenar_fatores_risco",
                "montante_minimo_emissao",
                "investimento_minimo_cpf_cnpj",
                "investimento_minimo_inst",
                "investimento_maximo_cpf_cnpj",
                "investimento_maximo_inst",
                "historico_cotacao_bolsa",
                "fator_proporcao_dp",
                "diluicao_economica_novas_emissoes",
                "criterio_rateio",
                "carteira_fundos_kinea_intrag",
                "breve_historico_gestor",
                "percentual_oferta_institucional",
                "volume_base_emissao",
                "chamada_capital_ipca"
            ]
        }
    }

    try:
        completion = openai.ChatCompletion.create(
            engine="gpt-4.1-mini",  
            messages=message_text,
            temperature=0.1,
            top_p=0.95,
            tools=[{"type": "function", "function": func}],
            tool_choice={"type": "function", "function": {"name": func["name"]}}
        )

        responseText = completion.to_dict()['choices'][0]['message']
        arguments_str = responseText['tool_calls'][0]["function"]["arguments"]
        
        # Parse do JSON retornado pela função
        dados_extraidos = json.loads(arguments_str)
        
        return dados_extraidos
        
    except json.JSONDecodeError as e:
        print(f"Erro ao fazer parse do JSON: {e}")
        return {"erro": "Falha ao processar resposta da IA - JSON inválido"}
        
    except KeyError as e:
        print(f"Erro de estrutura na resposta: {e}")
        return {"erro": "Falha na estrutura da resposta da IA"}
        
    except Exception as e:
        print(f"Erro geral: {e}")
        return {"erro": f"Erro inesperado: {str(e)}"}

# função para quando a LLM falhar em extrair algum dado importante.
# o fallback faz uma segunda busca no texto quando ele falha em encontrar a informação de primeira no prospecto. 
# campos aqui que eu considerei importantes e que apresentaram falhas as vezes na extração:
# - taxa de distribuição da emissão, tabela de ativos do fundo, 


#def fallback(campo, texto)


def main():
    lista_pdfs = [
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea2.pdf",
        "/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea3.pdf"
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea4.pdf",
    ]
    
    # Lista para armazenar todos os dados extraídos
    todos_dados = []
    
    for pdf in lista_pdfs:
        texto1, texto2 = dividir_pdf_em_duas_partes(pdf)
        dados1 = extrai_infos(texto1)
        dados2 = extrai_infos(texto2)

        dados_finais = combinar_dados(dados1, dados2)
        todos_dados.append(dados_finais)

        for chave, valor in dados_finais.items():
            print(f"{chave}: {valor}")
    
    if todos_dados:
        inserir_dados_databricks(todos_dados)
    else:
        print("Nenhum dado foi extraído com sucesso.")        

def combinar_dados(dados_parte1, dados_parte2):
    """
    Combina os dados de duas extrações, priorizando a primeira parte
    e usando a segunda parte apenas para preencher campos vazios.
    """
    dados_finais = dados_parte1.copy()
    
    campos_preenchidos = 0
    
    for chave, valor_parte1 in dados_parte1.items():
        # Se o campo está vazio na primeira parte, tentar preencher com a segunda
        if not valor_parte1.strip():  # Campo vazio ou só espaços
            if chave in dados_parte2 and dados_parte2[chave].strip():
                dados_finais[chave] = dados_parte2[chave]
                campos_preenchidos += 1
                print(f"  → Campo '{chave}' preenchido com dados da segunda parte")
    
    print(f"Total de campos preenchidos pela segunda parte: {campos_preenchidos}")
    
    return dados_finais
def inserir_dados_databricks(lista_dados):
    """
    Insere os dados extraídos na tabela do Databricks
    """
    print(f"\nInserindo {len(lista_dados)} registros na tabela...")
    
    # Definir o schema da tabela
    schema = StructType([
        StructField("cnpj", StringType(), True),
        StructField("data_emissao", StringType(), True),
        StructField("qt_emissoes", StringType(), True),
        StructField("nome_fundo", StringType(), True),
        StructField("valor_cota_emissao", StringType(), True),
        StructField("direito_preferencia_sobras_montante_adicional", StringType(), True),
        StructField("taxa_distribuicao_emissao", StringType(), True),
        StructField("tabela_ativos_fundo", StringType(), True),
        StructField("sumario_experiencia_socios", StringType(), True),
        StructField("quantidade_cotas_emissao", StringType(), True),
        StructField("quantidade_cotas_adicionais_emissao", StringType(), True),
        StructField("publico_alvo", StringType(), True),
        StructField("obs_publico_alvo", StringType(), True),
        StructField("procuracao_AGE", StringType(), True),
        StructField("planilha_custos", StringType(), True),
        StructField("ordenar_fatores_risco", StringType(), True),
        StructField("montante_minimo_emissao", StringType(), True),
        StructField("investimento_minimo_cpf_cnpj", StringType(), True),
        StructField("investimento_minimo_inst", StringType(), True),
        StructField("investimento_maximo_cpf_cnpj", StringType(), True),
        StructField("investimento_maximo_inst", StringType(), True),
        StructField("historico_cotacao_bolsa", StringType(), True),
        StructField("fator_proporcao_dp", StringType(), True),
        StructField("diluicao_economica_novas_emissoes", StringType(), True),
        StructField("criterio_rateio", StringType(), True),
        StructField("carteira_fundos_kinea_intrag", StringType(), True),
        StructField("breve_historico_gestor", StringType(), True),
        StructField("percentual_oferta_institucional", StringType(), True),
        StructField("volume_base_emissao", StringType(), True),
        StructField("chamada_capital_ipca", StringType(), True)
    ])
    
    # Garantir que todos os campos da tabela estão presentes
    dados_para_inserir = []
    for dados in lista_dados:
        dados_completos = {}
        for field in schema.fields:
            dados_completos[field.name] = dados.get(field.name, "")
        dados_para_inserir.append(dados_completos)
    
    # Criar DataFrame e inserir
    df = spark.createDataFrame(dados_para_inserir, schema)
    df.write.mode("append").saveAsTable("desafio_kinea.prospecto_fundos.extracao_prospectos_kinea_v2")
    
    print(f"✅ {len(lista_dados)} registros inseridos com sucesso!")
        
           
if __name__ == "__main__":
    main()


In [0]:
import fitz
import base64
import re
import os
import openai
import tempfile
import json
from datetime import datetime
from functools import lru_cache
from json import dumps
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SparkSession

# Configuração da API Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2025-01-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea', 'gpt-key')

def dividir_pdf_em_duas_partes(caminho_pdf):
    doc = fitz.open(caminho_pdf)
    total_paginas = len(doc)
    metade = total_paginas // 2

    texto_parte1 = ""
    texto_parte2 = ""

    for i in range(total_paginas):
        texto = doc[i].get_text()
        if i < metade:
            texto_parte1 += texto
        else:
            texto_parte2 += texto

    doc.close()
    return texto_parte1.strip(), texto_parte2.strip()

# A função a seguir:
# - recebe um texto (no caso o PDF convertido)
# - retorna um dicionário com as informações extraídas.

def extrai_infos(texto):
    prompt = f"""Role: Act as a highly specialized AI document analyst trained in processing and extracting structured data from official financial documents, specifically related to Investment Funds in Brazil.

    Context:
    You will receive the full text of a regulatory or offering document related to a quota issuance (e.g., prospecto). These documents are usually published in PDF format and contain financial, legal, and operational details about a fund offering.

    Your Objective:
    Extract ** this specific fields** from the document content. Each field corresponds to a relevant piece of information necessary for evaluating the offering. If multiple mentions of the same data point exist, prioritize:
    1. The version labeled as definitive or part of the final offer terms (e.g., \"condições da oferta\").
    2. The most recent or prominent value.
    3. If you are not sure of the information, it's better to return an empty string.
    

    Output Instructions:
    – **Do not infer or fabricate information**. If a field is not explicitly mentioned or lacks clarity, return an **empty string** `\"\"`.
    – All numerical values must use the **international format**: use `.` (dot) for decimals and **no thousand separators**.
    – Return the result as a **strictly valid JSON object**, conforming exactly to the schema below.
    – **Do not include any explanatory text**, headers, comments, or notes outside the JSON object.
    – The order of keys in the output must match the list below.
    – If a field is a list or summary, return a string.

    
    Constraints:
    – Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string.
    – Use Brazilian Portuguese language context for comprehension but ensure numeric formatting adheres to international standards.
    – Be precise and concise. Avoid speculative language.
    

    You are expected to handle varied writing styles and formatting typically found in official financial documents and maintain high accuracy under ambiguity.
    Informações a extrair (nessa ordem):
        1. CNPJ do fundo
        2. Data de emissão do fundos
        3. Quantidade de emissões que aquele fundo já passou
        4. Nome do Fundo
        5. Valor da Cota da Emissão
        6. Se existe direito de preferência, sobras ou montante adicional
        7. Taxa de distribuição da Emissão
        8. Tabela de ativos do fundo
        9. Sumário de experiência dos sócios
        10. Quantidade de cotas da Emissão
        11. Quantidade de cotas adicionais na Emissão
        12. Público-alvo
        13. Procuração para AGE (se necessária)
        14. Planilha de custos
        15. Ordenação dos fatores de risco
        16. Montante mínimo da Emissão
        17. Investimento mínimo por CPF/CNPJ
        18. Investimento mínimo para investidores institucionais
        19. Investimento máximo por CPF/CNPJ
        20. Investimento máximo para investidores institucionais
        21. Histórico de cotação em bolsa (mínima, média e máxima)
        22. Fator de proporção para o Direito de Preferência (DP)
        23. Diluição econômica em novas emissões
        24. Critério de rateio
        25. Carteira de fundos Kinea com Intrag
        26. Breve histórico do gestor
        27. % da oferta destinada ao público institucional
        28. Volume base da Emissão
    """
    message_text = [
        {'role': 'system', 'content': prompt},
        {"role": "user", "content": texto}
    ]

    func = {
        'name': 'extrair_informacao_pdf',
        'description': 'Função para extrair os valores do PDF.',
        'parameters': {
            "type": "object",
            "properties": {
                "cnpj": {
                "type": "string",
                "description": "CNPJ do fundo. Extrair informação no formato padrão brasileiro (ex: \"60.431.592/0001-28\")."
                },
                "data_emissao": {
                "type": "string",
                "description": "Data de emissão do fundo. A informação pode estar no texto com formato ex: 'A data deste Prospecto Definitivo é 23 de junho de 2025'. Extraia a a data no formato YYYY-MM-DD (ex: \"2025-06-23\")."
                },
                "qt_emissoes": {
                "type": "string",
                "description": "Quantidade de emissões que aquele fundo já passou. Informe como número (ex.: \"1\", \"2\", etc.). A informação pode estar no formato 'PROSPECTO DEFINITIVO DE DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 1ª (PRIMEIRA) EMISSÃO...' ou então 'DISTRIBUIÇÃO PÚBLICA PRIMÁRIA DE COTAS DA 4ª (QUARTA) EMISSÃO...'." 
                },
                "nome_fundo": {
                "type": "string",
                "description": "Extrair o nome do fundo"
                },
                "valor_cota_emissao": {
                "type": "string",
                "description": """Preço unitário de subscrição por cota na nova emissão. Informe como número (ex.: \"98.45\") ou texto vazio se não disponível.
                Consultar texto introdutório do prospecto. Informação pode estar no formato: "...nominativas e escriturais, da 4ª Emissão ("4ª Emissão" e "Novas Cotas"), pelo valor unitário de R$ 101,26 (cento e um reais e vinte e seis centavos), correspondente ao valor patrimonial das cotas da Classe... Caso não exista, retornar string vazia."""
                },
                "direito_preferencia_sobras_montante_adicional": {
                "type": "string",
                "description": """Indique se há ou não direito de preferência, possibilidade de sobras ou de montante adicional. Ex.: \"Sim\" ou \"Não\".
                A informação pode estar na seção com nome "Informações sobre a existência de direito de preferência na subscrição de novas cotas" ou similar. Caso não exista, retornar string vazia."""
                },
                "taxa_distribuicao_emissao": {
                "type": "string",
                "description": "Percentual ou valor fixo da taxa de distribuição primária (remuneração do coordenador líder). Ex.: \"3.0%\". A informação pode estar numa tabela seguida do número por extenso. Extraia o valor do percentual em relação ao valor da cota ou então o percentual em relação ao montante total. Ex: 2,00% (dois por cento) . Caso não exista, retornar string vazia."
                },
                "tabela_ativos_fundo": {
                "type": "string",
                "description": """Cópia ou referência resumida da tabela/lista dos principais ativos que compõem a carteira-alvo ou carteira atual do fundo.Liste os ativos somente com os nomes deles (seguidos da sigla se tiver ex: CRI) separados por vírgula. Não é necessário numerar. A informação pode estar na seção 'Destinação dos Recursos' ou similar. A informação pode estar no formato '...este sentido, os Ativos nos quais o Fundo poderá investir são: (i) Certificados de Recebíveis Imobiliários ("CRI"); (ii) cotas de fundos de investimento imobiliário ("FII"); (iii) debêntures ("Debêntures") emitidas por emissores devidamente autorizados nos termos da regulamentação aplicável, e cujas atividades preponderantes sejam permitidas aos FII... Caso não exista, retornar string vazia."""
                },
                "sumario_experiencia_socios": {
                "type": "string",
                "description": "Síntese da experiência profissional dos sócios/gestores apresentada no documento. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_emissao": {
                "type": "string",
                "description": "Quantidade total de cotas ofertadas na emissão base (sem considerar lotes adicionais). Ex.: \"5.000.000\". Consultar texto inicial do prospecto. Caso não exista, retornar string vazia."
                },
                "quantidade_cotas_adicionais_emissao": {
                "type": "string",
                "description": "Quantidade de cotas do lote adicional ou suplementar, se previsto. Consultar texto inicial do prospecto. A informação pode estar no formato '... Os Ofertantes, nos termos e conforme os limites estabelecidos no artigo 50 da Resolução CVM nº 160, com a prévia concordância do Coordenador Líder (conforme abaixo definido), poderão optar por distribuir um volume adicional de até 25% (vinte e cinco por cento) da quantidade máxima de Novas Cotas inicialmente ofertadas, ou seja, até 1.234.445 (um milhão, duzentas e trinta e quatro mil, quatrocentas e quarenta e cinco) Novas Cotas...' . Caso não exista, retornar string vazia."
                },
                "publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta. A informação pode estar na seção 'Identificação do público alvo' ou similar. Retorne entre as opções possíveis: \"Investidores em geral\" , \"Investidores qualificados\", \"Investidores Institucionais\". Se a informação não está presente no texto, retorne \"Investidores em geral\" como padrão. "
                },
                "obs_publico_alvo": {
                "type": "string",
                "description": "Descrição textual do público-alvo da oferta. A informação pode estar na seção 'Identificação do público alvo' ou similar. Consulte esse fragmento de texto para extrair se, além de Investidores em geral/qualificados/institucionais, existem outras restrições. Retorne somente os casos especiais (ex: \"Clientes Itaú Unibanco\").Caso não exista, retornar string vazia."
                },
                "procuracao_AGE": {
                "type": "string",
                "description": "Indicação se o documento contém modelo de procuração para AGE ou se não é necessária. A informação pode estar na seção 'Indicar a eventual possibilidade de destinação dos recursos a quaisquer ativos em relação às quais possa haver conflito de interesse, informando as aprovações necessárias existentes e/ou a serem obtidas, incluindo nesse caso nos fatores de risco, explicação objetiva sobre a falta de transparência na formação dos preços destas operações' ou similar. Ex.: \"Modelo de procuração incluso\" ou \"Não aplicável\".Caso não exista, retornar string vazia."
                },
                "planilha_custos": {
                "type": "string",
                "description": "Referência ou resumo da planilha de custos da oferta (taxas, despesas, previsão de custos). A informação pode estar na seção 'Demonstrativo do custo da distribuição, discriminando: a) a porcentagem em relação ao preço unitário de subscrição; b) a comissão de coordenação; c) a comissão de distribuição; d) a comissão de garantia de subscrição, se houver; e) outras comissões (especificar); f) os tributos incidentes sobre as comissões, caso estes sejam arcados pela classe de cotas; g) o custo unitário de distribuição; h) as despesas decorrentes do registro de distribuição; e i) outros custos relacionados' ou similar. Ex: \"Taxa de distribuição total 2.00%, comissão de coordenação 0.10%, comissão de distribuição 1.70%, advogados 0.04%, tributos advogados 0.00%, taxa registro CVM 0.04%, taxa registro Anbima 0.00%, taxa B3 distribuição padrão fixa 0.03%, taxa B3 distribuição padrão variável 0.04%, taxa B3 análise oferta 0.00%, taxa B3 listagem 0.00%, anúncio 0.00%, cartório 0.00%, outras despesas 0.02%.\"Caso não exista, retornar string vazia. "
                },
                "ordenar_fatores_risco": {
                "type": "string",
                "description": "Fatores de risco já em ordem apresentada ou indicação de onde ela se encontra. Indicar o nome do risco seguido da sua intensidade ex: Risco de Crédito (Maior). Não use listas, somente strings simples.A informação pode estar na seção 'Fatores de Risco' ou similar.Caso não exista, retornar string vazia."
                },
                "montante_minimo_emissao": {
                "type": "string",
                "description": "Valor mínimo da oferta para que a emissão seja considerada válida (montante mínimo de liquidação). A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_cpf_cnpj": {
                "type": "string",
                "description": "Valor mínimo de subscrição por investidor pessoa física ou jurídica (CPF/CNPJ).A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_minimo_inst": {
                "type": "string",
                "description": "Valor mínimo de subscrição exigido para investidores institucionais, se diferente. A informação pode estar na seção  'Identificação do público-alvo' ou similar. Consulte para saber se investidores institucionais são contemplados. Se sim, extrair o valor mínimo de investimento para esse caso. Caso não exista, retornar string vazia."
                },
                "investimento_maximo_cpf_cnpj": {
                "type": "string",
                "description": "Valor máximo permitidos por investidor pessoa física ou jurídica (CPF/CNPJ), se houver restrição. A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "investimento_maximo_inst": {
                "type": "string",
                "description": "Valor máximo permitido para investidores institucionais, se houver. A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia."
                },
                "historico_cotacao_bolsa": {
                "type": "string",
                "description": "Valores mínima, média e máxima da cota em bolsa conforme apresentado (ex.: \"Mín 90.10 / Méd 94.50 / Máx 98.00\").A informação pode estar na seção 'Cotação em bolsa de valores ou mercado de balcão dos valore mobiliários a serem distribuídos, inclusive no exterior' ou similar.Caso não exista, retornar string vazia."
                },
                "fator_proporcao_dp": {
                "type": "string",
                "description": "Fator de proporção do Direito de Preferência (DP). A informação pode estar na seção 'Informações sobre a existência de direito de preferência na subscrição de novas cotas'. Extraia a proporção, se existir, no mesmo formato descrito no documento.Caso não exista, retornar string vazia."
                },
                "diluicao_economica_novas_emissoes": {
                "type": "string",
                "description": "Percentual estimado de diluição econômica para cotistas atuais após a nova emissão, ou texto explicativo.Caso não exista, retornar string vazia."
                },
                "criterio_rateio": {
                "type": "string",
                "description": "Critério de rateio em caso de excesso de demanda. A informação pode estar na seção 'Critério de Rateio' ou similar. Ex.: \"Pro rata\" ou \"Prioridade investidores profissionais\".Caso não exista, retornar string vazia."
                },
                "carteira_fundos_kinea_intrag": {
                "type": "string",
                "description": "Informação específica sobre a carteira de fundos Kinea com Intrag, se mencionada.Caso não exista, retornar string vazia."
                },
                "breve_historico_gestor": {
                "type": "string",
                "description": "Resumo do histórico da gestora ou do gestor responsável conforme descrito no documento.Caso não exista, retornar string vazia."
                },
                "percentual_oferta_institucional": {
                "type": "string",
                "description": "Percentual da oferta reservado ao público institucional. Ex.: \"30%\".Caso não exista, retornar string vazia."
                },
                "volume_base_emissao": {
                "type": "string",
                "description": "Montante financeiro (R$) correspondente ao volume base da emissão sem considerar lote adicional.A informação pode estar no formato ' perfazendo o valor total de até R$ 352.539.180,00 (trezentos e cinquenta e dois milhões, quinhentos e trinta e nove mil, cento e oitenta reais), considerando o Valor da Cota (“Volume Total da Oferta”), ' .Caso não exista, retornar string vazia."
                },
                "chamada_capital_ipca": {
                "type": "string",
                "description": "Descrição de cláusula de chamada de capital com atualização pelo IPCA, se existente. Caso não exista, retornar string vazia."
                }
            },
            "required": [
                "cnpj",
                "data_emissao",
                "qt_emissoes",
                "nome_fundo"
                "valor_cota_emissao",
                "direito_preferencia_sobras_montante_adicional",
                "taxa_distribuicao_emissao",
                "tabela_ativos_fundo",
                "sumario_experiencia_socios",
                "quantidade_cotas_emissao",
                "quantidade_cotas_adicionais_emissao",
                "publico_alvo",
                "obs_publico_alvo",
                "procuracao_AGE",
                "planilha_custos",
                "ordenar_fatores_risco",
                "montante_minimo_emissao",
                "investimento_minimo_cpf_cnpj",
                "investimento_minimo_inst",
                "investimento_maximo_cpf_cnpj",
                "investimento_maximo_inst",
                "historico_cotacao_bolsa",
                "fator_proporcao_dp",
                "diluicao_economica_novas_emissoes",
                "criterio_rateio",
                "carteira_fundos_kinea_intrag",
                "breve_historico_gestor",
                "percentual_oferta_institucional",
                "volume_base_emissao",
                "chamada_capital_ipca"
            ]
        }
    }

    try:
        completion = openai.ChatCompletion.create(
            engine="gpt-4.1-mini",  
            messages=message_text,
            temperature=0.1,
            top_p=0.95,
            tools=[{"type": "function", "function": func}],
            tool_choice={"type": "function", "function": {"name": func["name"]}}
        )

        responseText = completion.to_dict()['choices'][0]['message']
        arguments_str = responseText['tool_calls'][0]["function"]["arguments"]
        
        # Parse do JSON retornado pela função
        dados_extraidos = json.loads(arguments_str)
        
        return dados_extraidos
        
    except json.JSONDecodeError as e:
        print(f"Erro ao fazer parse do JSON: {e}")
        return {"erro": "Falha ao processar resposta da IA - JSON inválido"}
        
    except KeyError as e:
        print(f"Erro de estrutura na resposta: {e}")
        return {"erro": "Falha na estrutura da resposta da IA"}
        
    except Exception as e:
        print(f"Erro geral: {e}")
        return {"erro": f"Erro inesperado: {str(e)}"}

# função para quando a LLM falhar em extrair algum dado importante.
# o fallback faz uma segunda busca no texto quando ele falha em encontrar a informação de primeira no prospecto. A nova busca vai atrás exatamente da informação faltante.
# campos aqui que eu considerei importantes e que apresentaram falhas as vezes na extração:
# - taxa de distribuição da emissão, tabela de ativos do fundo, fatores de risco, montante minimo de emissao


import time

# Prompt templates específicos por campo
_FOCUSED_PROMPTS = {
    "taxa_distribuicao_emissao": """
Você recebeu o texto abaixo (trecho de um prospecto de fundo de investimento).
Objetivo: extrair **apenas a taxa de distribuição da emissão primária** (remuneração do coordenador líder), no formato percentual com ponto decimal e com símbolo de porcentagem, ex: "2.00%".
Regras: 
- Não infira nada que não esteja explícito.
- Se houver múltiplas menções, prefira a que esteja descrita como "taxa de distribuição", "remuneração do coordenador líder" ou "taxa de distribuição total".
- Se não achar, retorne string vazia.

Texto:
\"\"\"{texto}\"\"\"
""",
    "tabela_ativos_fundo": """
Você recebeu o texto abaixo (trecho de um prospecto de fundo de investimento).
Objetivo: extrair os principais ativos que compõem a carteira do fundo, retornando **somente os nomes ou siglas** (ex: CRI, FII, Debêntures) separados por vírgula, sem duplicatas.
Regras:
- Liste os ativos citados no estilo "Ativos nos quais o Fundo poderá investir são: ..." ou em seções similares.
- Se não houver definição clara, retorne string vazia.

Texto:
\"\"\"{texto}\"\"\"
""",
    "ordenar_fatores_risco": """
Você recebeu o texto abaixo (trecho de um prospecto de fundo de investimento).
Objetivo: identificar os fatores de risco listados e, se possível, associar a eles uma intensidade quando apresentada (ex: "Risco de Crédito (Maior)", "Risco de Mercado - Moderado"), e devolver uma única string com os riscos formatados como: Nome do risco (Intensidade), separados por vírgula.
Regras:
- Use a ordem em que aparecem no documento.
- Se não houver intensidade explícita, apenas coloque o nome do risco sem parênteses.
- Se não encontrar nada, retorne string vazia.

Texto:
\"\"\"{texto}\"\"\"
""",
    "volume_base_emissao": """
Você recebeu o texto abaixo (trecho de um prospecto de fundo de investimento).
Objetivo: extrair o **montante mínimo da emissão**: Montante financeiro (R$) correspondente ao volume base da emissão sem considerar lote adicional.A informação pode estar no formato ' perfazendo o valor total de até R$ 352.539.180,00 (trezentos e cinquenta e dois milhões, quinhentos e trinta e nove mil, cento e oitenta reais), considerando o Valor da Cota (“Volume Total da Oferta”) - nesse caso retornar 352539180, ' .Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string. Caso não exista, retornar string vazia."

Texto:
\"\"\"{texto}\"\"\"
""",
    "montante_minimo_emissao": """
Você recebeu o texto abaixo (trecho de um prospecto de fundo de investimento).
Objetivo: extrair o **montante mínimo da emissão**: Valor mínimo da oferta para que a emissão seja considerada válida (montante mínimo de liquidação). A informação pode estar na seção 'Requisitos ou exigências mínimas de investimento, caso existam' ou similar.Caso não exista, retornar string vazia. A informação pode estar no formato 'A Oferta terá o valor mínimo de R$ 20.145.096,00 (vinte milhões, cento e quarenta e cinco mil e noventa e seis reais)'.Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string. "

Texto:
\"\"\"{texto}\"\"\"
""",
"quantidade_cotas_adicionais_emissao": """
Você recebeu o texto abaixo (trecho de um prospecto de fundo de investimento).
Objetivo: extrair a **quantidade_cotas_adicionais_emissao**: Quantidade de cotas do lote adicional ou suplementar, se previsto. Consultar texto inicial do prospecto. A informação pode estar no formato '... Os Ofertantes, nos termos e conforme os limites estabelecidos no artigo 50 da Resolução CVM nº 160, com a prévia concordância do Coordenador Líder (conforme abaixo definido), poderão optar por distribuir um volume adicional de até 25% (vinte e cinco por cento) da quantidade máxima de Novas Cotas inicialmente ofertadas, ou seja, até 1.234.445 (um milhão, duzentas e trinta e quatro mil, quatrocentas e quarenta e cinco) Novas Cotas...' .Do not include monetary symbols (e.g., R$) in the values; return only the pure numeric string. Caso não exista, retornar string vazia. "

Texto:
\"\"\"{texto}\"\"\"
""",
}

def _chamada_llm_focalizada(campo, texto):
    """
    Faz chamada reduzida à LLM para extrair um único campo específico.
    """
    prompt_template = _FOCUSED_PROMPTS.get(campo)
    if not prompt_template:
        return ""

    filled = prompt_template.format(texto=texto)

    messages = [
        {"role": "system", "content": f"Você é um extrator especializado e precisa devolver o valor de '{campo}'."},
        {"role": "user", "content": filled}
    ]

    func = {
        "name": "extrair_campo_especifico",
        "description": f"Extrai o campo {campo} do texto.",
        "parameters": {
            "type": "object",
            "properties": {
                campo: {
                    "type": "string",
                    "description": f"Valor extraído para {campo}"
                }
            },
            "required": [campo]
        }
    }

    attempt = 0
    while attempt < 2:  # duas tentativas
        try:
            completion = openai.ChatCompletion.create(
                engine="gpt-4.1-mini",
                messages=messages,
                temperature=0.1,
                top_p=0.95,
                tools=[{"type": "function", "function": func}],
                tool_choice={"type": "function", "function": {"name": func["name"]}}
            )
            response = completion.to_dict()['choices'][0]['message']
            if 'tool_calls' in response and response['tool_calls']:
                arguments_str = response['tool_calls'][0]['function']['arguments']
                dados = json.loads(arguments_str)
                valor = dados.get(campo, "")
                if isinstance(valor, str):
                    return valor.strip()
                else:
                    return str(valor).strip()
            else:
                content = response.get("content", "").strip()
                if content:
                    return content
        except Exception:
            time.sleep(1.5 ** attempt)
        attempt += 1
    return ""

def fallback(campo, texto_1a_metade, texto_2a_metade):
    """
    Primeiro tenta heurísticas locais (podem ser adicionadas aqui).
    Se não encontrar, faz segunda chamada da LLM especializada naquele campo.
    """
    # combinação dos textos para contexto
    texto = f"{texto_1a_metade}\n{texto_2a_metade}"
    texto = re.sub(r"\s+", " ", texto)

    # (Opcional) aqui você pode inserir heurísticas locais antes de chamar a LLM.
    # Por simplicidade, vamos direto para a LLM focalizada se o campo for dos críticos.
    if campo in _FOCUSED_PROMPTS:
        tentativa_llm = _chamada_llm_focalizada(campo, texto)
        return tentativa_llm or ""
    return ""




def main():
    lista_pdfs = [
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea2.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea3.pdf"
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea4.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/KPRM_Prospecto.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/KFOF_Prospecto_12-2023.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/KNRI_Prospecto_2024-03.pdf",
        "/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/KNCR11_Prospecto_10-2024.pdf",
        #"/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/Prospecto-10a-Emissao-KNIP.pdf"
        "/Volumes/desafio_kinea/prospecto_fundos/arquivos-prospectos/arquivos-pdf/prospeckinea2.pdf"
    ]

    campos_criticos = [
        "taxa_distribuicao_emissao",
        "tabela_ativos_fundo",
        "ordenar_fatores_risco",
        "montante_minimo_emissao",
        "volume_base_emissao",
        "quantidade_cotas_adicionais_emissao"
    ]

    todos_dados = []

    for pdf in lista_pdfs:
        print(f"\nProcessando PDF: {pdf}")
        texto1, texto2 = dividir_pdf_em_duas_partes(pdf)

        dados1 = extrai_infos(texto1)
        dados2 = extrai_infos(texto2)

        dados_finais = combinar_dados(dados1, dados2)

        # fallback via LLM para os campos críticos ainda vazios
        for campo in campos_criticos:
            if not dados_finais.get(campo, "").strip():
                tentativa = fallback(campo, texto1, texto2)
                if tentativa and tentativa.strip():
                    dados_finais[campo] = tentativa.strip()
                    print(f"  → Campo '{campo}' preenchido via fallback LLM: {tentativa.strip()}")

        # impressão para debug
        for chave, valor in dados_finais.items():
            print(f"{chave}: {valor}")

        todos_dados.append(dados_finais)

    if todos_dados:
        inserir_dados_databricks(todos_dados)
    else:
        print("Nenhum dado foi extraído com sucesso.")

def combinar_dados(dados_parte1, dados_parte2):
    """
    Combina os dados de duas extrações, priorizando a primeira parte
    e usando a segunda parte apenas para preencher campos vazios.
    """
    dados_finais = dados_parte1.copy()
    
    campos_preenchidos = 0
    
    for chave, valor_parte1 in dados_parte1.items():
        # Se o campo está vazio na primeira parte, tentar preencher com a segunda
        if not valor_parte1.strip():  # Campo vazio ou só espaços
            if chave in dados_parte2 and dados_parte2[chave].strip():
                dados_finais[chave] = dados_parte2[chave]
                campos_preenchidos += 1
                print(f"  → Campo '{chave}' preenchido com dados da segunda parte")
    
    print(f"Total de campos preenchidos pela segunda parte: {campos_preenchidos}")
    
    return dados_finais
def inserir_dados_databricks(lista_dados):
    """
    Insere os dados extraídos na tabela do Databricks
    """
    print(f"\nInserindo {len(lista_dados)} registros na tabela...")
    
    # Definir o schema da tabela
    schema = StructType([
        StructField("cnpj", StringType(), True),
        StructField("data_emissao", StringType(), True),
        StructField("qt_emissoes", StringType(), True),
        StructField("nome_fundo", StringType(), True),
        StructField("valor_cota_emissao", StringType(), True),
        StructField("direito_preferencia_sobras_montante_adicional", StringType(), True),
        StructField("taxa_distribuicao_emissao", StringType(), True),
        StructField("tabela_ativos_fundo", StringType(), True),
        StructField("sumario_experiencia_socios", StringType(), True),
        StructField("quantidade_cotas_emissao", StringType(), True),
        StructField("quantidade_cotas_adicionais_emissao", StringType(), True),
        StructField("publico_alvo", StringType(), True),
        StructField("obs_publico_alvo", StringType(), True),
        StructField("procuracao_AGE", StringType(), True),
        StructField("planilha_custos", StringType(), True),
        StructField("ordenar_fatores_risco", StringType(), True),
        StructField("montante_minimo_emissao", StringType(), True),
        StructField("investimento_minimo_cpf_cnpj", StringType(), True),
        StructField("investimento_minimo_inst", StringType(), True),
        StructField("investimento_maximo_cpf_cnpj", StringType(), True),
        StructField("investimento_maximo_inst", StringType(), True),
        StructField("historico_cotacao_bolsa", StringType(), True),
        StructField("fator_proporcao_dp", StringType(), True),
        StructField("diluicao_economica_novas_emissoes", StringType(), True),
        StructField("criterio_rateio", StringType(), True),
        StructField("carteira_fundos_kinea_intrag", StringType(), True),
        StructField("breve_historico_gestor", StringType(), True),
        StructField("percentual_oferta_institucional", StringType(), True),
        StructField("volume_base_emissao", StringType(), True),
        StructField("chamada_capital_ipca", StringType(), True)
    ])
    
    # Garantir que todos os campos da tabela estão presentes
    dados_para_inserir = []
    for dados in lista_dados:
        dados_completos = {}
        for field in schema.fields:
            dados_completos[field.name] = dados.get(field.name, "")
        dados_para_inserir.append(dados_completos)
    
    # Criar DataFrame e inserir
    df = spark.createDataFrame(dados_para_inserir, schema)
    df.write.mode("append").saveAsTable("desafio_kinea.prospecto_fundos.extracao_prospectos_kinea_v2")
    
    print(f"✅ {len(lista_dados)} registros inseridos com sucesso!")
        
           
if __name__ == "__main__":
    main()
